# script for “A mechanistic model to link technical specifications of vehicle end-of-life treatment with the potential of closed-loop recycling of post-consumer scrap alloys”

### Import packages

In [ ]:
import xlrd,xlwt
import numpy as np
from numpy.linalg import inv
from numpy.linalg import norm
import pandas as pd
import collections
from copy import deepcopy
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
import itertools
import matplotlib.pyplot as plt
from functools import reduce

#set random seed
np.random.seed(123)

### Time-series module

In [ ]:
def MFA_MOD(MFA_time_series_file,scenario_num):
    """This module provides data regarding # of vehicles
    ## Input variables:
    -MFA_time_series_file: a workbook contains data sheets of time series data
    -scenario_num: scenario number that reflects different assumptions regarding the MFA scenarios
    ## Output variables
    -num_new_vehicles_dict: a dict that contains the number of new vehicles to be manufactured by category each year, in the format of {"vehicle type":[{year1:amount1},{year1:amount1}..]}
    -num_retired_vehicles_dict: a dict that contains the number of retired vehicles by category each year, in the format of {"vehicle type":[{year1:amount1},{year1:amount1}..]}
    -time_span_tuple: a tuple contains the starting and ending years of the period
    """

    
    """load data file"""
    xls_file=pd.ExcelFile(MFA_time_series_file)
    #load inflow (new vehicle demand) tabs
    inflow_all_sheets=collections.defaultdict()
    inflow_all_sheets[1]=pd.read_excel(xls_file,"consolidated_inflow_s1").iloc[:,:7] #only keep the columns of interest
    inflow_all_sheets[2]=pd.read_excel(xls_file,"consolidated_inflow_s2").iloc[:,:7]
    inflow_all_sheets[3]=pd.read_excel(xls_file,"consolidated_inflow_s3").iloc[:,:7]
    #load outflow (vehicle retirement) tabs
    outflow_all_sheets=collections.defaultdict()
    outflow_all_sheets[1]=pd.read_excel(xls_file,"consolidated_outflow_s1").iloc[:,:7]
    outflow_all_sheets[2]=pd.read_excel(xls_file,"consolidated_outflow_s2").iloc[:,:7]
    outflow_all_sheets[3]=pd.read_excel(xls_file,"consolidated_outflow_s3").iloc[:,:7]
    
    
    """assign data to vehicle demand and retirement"""
    ##create output variables
    num_new_vehicles_dict=collections.defaultdict(list)
    num_retired_vehicles_dict=collections.defaultdict(list)
    time_span_tuple=(int(inflow_all_sheets[2].iloc[0,0]),int(inflow_all_sheets[2].iloc[-1,0]))
    
    ##assignment based on the scenario specified by the user
    for col_name in inflow_all_sheets[scenario_num].columns.values[1:]:
        for index,year in enumerate (range (time_span_tuple[0],time_span_tuple[1]+1)):
            num_new_vehicles_dict[col_name].append({year:inflow_all_sheets[scenario_num].loc[index,col_name]})
            num_retired_vehicles_dict[col_name].append({year:outflow_all_sheets[scenario_num].loc[index,col_name]})                                  
    

    return time_span_tuple,num_new_vehicles_dict,num_retired_vehicles_dict
    
    


### Dismantling function

In [ ]:
def Dismantling (v_c_tensor,v_s_tensor,mat_compnt_tensor):
    """This function calculates material composition (by COMPONENT) after dismantling
        **this version of the function does tensor operation which 'carries' the third dismension of vehicle type
    ## Input variables:
    -v_c_tensor: a 3D tensor contains vectors containing mass of the components (j) of a specific type of vehicle (1) among c vehicle types, in total mass (kg), (j,1,c)
    -v_s_tensor: a 3D tensor contains vectors containing the dismantling rate of each component (rest is left to "car hulk"), corresponding to v_c_tensor, in %, (j,1,c)
        note that "reuse/remanufacturing" (e.g., remanu of engine) is not considered; "dismantling rate" strictly refers to the portion that is "unwanted and shredded separately (from other components or hulk)"
    -mat_compnt_tensor: materials (row) by components (col) matrix that shows the material composition of each component, 
       in normalized mass (kg/kg), (n,j), for all vehicle types (c), (n,j,c)
    ## Output variables:
    -scrap_compnt_tensor: scrap materials (n) by component (j) matrix for vehicle types (c) that shows the material composition of each component for a given vehicle type
       This differs from "mat_com_tensor", as it is in total mass (kg), (n,j,c)
    -scrap_compnt_to_hulk_tensor: scrap materials (n) by component (j) that are not dismantled (will go to car hulk) for each vehicle type (c), in total mass (kg), (n,j,c)    
    """
    
    """Calculate the mass of component that is dismantled (the rest is left with hulk)"""
    ##element-wise multiplication to calculate the mass of compnt that will be shreded separately after dismantling or to be shredded as a whole (hulk)
    vc_vs_Hadamard_to_separate_shred=np.multiply(v_c_tensor,v_s_tensor)
    vc_vs_Hadamard_to_hulk_shred=np.multiply(v_c_tensor,1-v_s_tensor)
    ##diagonalize the results above
    #as np.diagnolize is too tricky to use for 3D tensor, I did the following instead
    #for those go to separate compnt shredding first
    vc_vs_Hadamard_to_separate_shred_diag=np.repeat(vc_vs_Hadamard_to_separate_shred[:,:,:],vc_vs_Hadamard_to_separate_shred.shape[0],axis=1)
    eye_matrix=np.eye(vc_vs_Hadamard_to_separate_shred_diag.shape[0])
    eye_matrix=np.repeat(eye_matrix[:,:,np.newaxis],vc_vs_Hadamard_to_separate_shred_diag.shape[2],axis=2)
    eye_tensor=eye_matrix
    vc_vs_Hadamard_to_separate_shred_diag=np.multiply(vc_vs_Hadamard_to_separate_shred_diag,eye_tensor)
    #for those go as a whole hulk
    vc_vs_Hadamard_to_hulk_shred_diag=np.repeat(vc_vs_Hadamard_to_hulk_shred[:,:,:],vc_vs_Hadamard_to_hulk_shred.shape[0],axis=1)
    vc_vs_Hadamard_to_hulk_shred_diag=np.multiply(vc_vs_Hadamard_to_hulk_shred_diag,eye_tensor)
   
    scrap_compnt_tensor=np.einsum('njc,jlc -> njc',mat_compnt_tensor,vc_vs_Hadamard_to_separate_shred_diag)
    scrap_compnt_to_hulk_tensor=np.einsum('njc,jlc -> njc',mat_compnt_tensor,vc_vs_Hadamard_to_hulk_shred_diag)
    
    return scrap_compnt_tensor,scrap_compnt_to_hulk_tensor    

### Shredding function

In [ ]:
def Shredding(to_be_shredded,AE_matrix,shredding_loss_rate,cross_mixing=True):
    """This function calculates the shredded alloys and their AE composition of each COMPONENT or CAR HULK
    ## Input variables
    -to_be_shredded: 
        "scrap_compnt_matrix": (1) scrap materials (row) by component (col) matrix that shows the material composition of each component, 
        in total mass (kg), (n,j) or (2) 3D tensor that has one more dimension (vehicle type) (n,j,c)
        OR
        "car hulk": (1) scrap materials (row) by car hulk (col=1) matrix that shows the material composition of car hulk, 
        in total mass (kg), (n,1) or (2) 3D tensor that has one more dimension (vehicle type) (n,1,c)
    -AE_matrix: alloys (row) by alloying elements (col) matrix that shows the AE composition of each alloy, in %, (n,z)
    -shredding_loss_rate: a list contains the shredding loss rate for each component (len=j) or car hulk (len=1)
    -cross_mixing: a boolean variable to indicate if cross_mixing is assumed, "True" by default
    ## Output variables
    -shredded_alloys_mix_by_compnt: a 3D tensor contains shredded scrap alloys (row) by AE composition (col), for each component (channel)
        in total mass (kg), (n,z,j)
    -shred_loss_to_others: a 3D tensor contains shredded scrap alloys (row) by AE composition (col), for each component (channel) that are lost
        in total mass (kg), (n,z,j)
    -to_be_shredded_copy: a copy of input variable "to_be_shredded" that will be used for creating "contribute_ratio" in EoL MOD later, (n,j) or (n,j,c)
    """
    
    """use a for loop to obtain shredded_alloys_mix for each component"""
    #initiate the tensor
    row,col,channel=(AE_matrix.shape[0],AE_matrix.shape[1],to_be_shredded.shape[1])
    shredded_alloys_mix_by_compnt=np.zeros((row,col,channel))
    shred_loss_to_others=np.zeros((row,col,channel))
    

    ##make a copy of "to_be_shredded"
    to_be_shredded_copy=deepcopy(to_be_shredded)


    ##check if cross-mixing is assumed:
    if cross_mixing:
        ##collapse the channel dimension (vehicle)
        to_be_shredded=np.sum(to_be_shredded,axis=2) #(n,j)        
        
    
    ##shredding
    for j in range(to_be_shredded.shape[1]):
        #first diagonalize the scrap alloy mix for each component, then dot product is with AE_matrix to get alloy by AE composition for this compnent
        temp=np.dot(np.diag(to_be_shredded[:,j]),AE_matrix) 
        shredded_alloys_mix_by_compnt[:,:,j]=temp*(1-shredding_loss_rate[j])
        shred_loss_to_others[:,:,j]=temp*shredding_loss_rate[j]

            
    return shredded_alloys_mix_by_compnt,shred_loss_to_others,to_be_shredded_copy

### Sorting&remelting function

In [ ]:
def Sorting_and_remelting(to_be_sorted,alloy_group_cutoff,TD_matrix,rand_sort_error=0.02):
    """This function calculates the intermediate scrap alloy by metal group and their AE composition of each COMPONENT 
    ## Input variables
    -to_be_sorted:
        shredded_alloys_mix_by_compnt: a 3D tensor contains shredded scrap alloys (row) by AE composition (col), 
            for each component OR car hulk (channel), in total mass (kg), (n,z,j)
    -TD_matrix: thermodynmic limit matrix that contains the yield of AE (col) during remelting of scrap allys of same group (row),
        in %, (y,z)
    -rand_sort_error: a random sorting error rate (scalar) that indicates the degree of imperfect sorting 
    -alloy_group_cutoff: a list contains tuple of (staring,ending) rows in the alloy-related matrix that belongs to a certain common metal group,
        len()= #of metal types = y
    ## Output variables
    -IntAlloy_by_compnt: a 3D tensor that contains intermediate scrap alloy by metal group (row), by AE composition (col), 
        for each component OR car hulk (channel), in total mass (kg), (y,z,j)
    -remelt_loss_to_others: a 3D tensor that contains input alloys that are LOST during remelting by metal group (row), by AE composition (col), 
        for each component OR car hulk (channel), in total mass (kg), (y,z,j)
    """

    
    """sort (and combine) scrap alloys by metal group"""

    ##initiate a "to_be_remelted" tensor of (y,z,j)
    row,col,channel=(TD_matrix.shape[0],TD_matrix.shape[1],to_be_sorted.shape[2])
    to_be_remelted=np.zeros((row,col,channel))
    #print ("shape of remelted: ", to_be_remelted.shape)
    
    for index,cutoff_tuple in enumerate(alloy_group_cutoff):
        to_be_remelted[index,:,:]=np.sum(to_be_sorted[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:],axis=0,keepdims=True) #collapse k rows to one row, while keep the other two dimeions unchanged
        
    if rand_sort_error>0.0:
        """rand_sort_error algorithm (assume equal probability for all alloy groups)"""
        ##First, set aside the portion of scrap alloy that may be mis-classified
        rand_set_aside=to_be_remelted*rand_sort_error #a 3D tensor, if error=0, then no mass is set aside
        to_be_remelted-=rand_set_aside

        ##create concentration tensor for the set-aside mass tensor
        rand_set_aside_conc=np.nan_to_num(rand_set_aside/np.sum(rand_set_aside,axis=1,keepdims=True))
        
        ##randomly shuffle the index of ALL metal groups
        temp_order_all=np.arange(rand_set_aside.shape[0]) #create a array of row index of rand_set_aside, e.g., (0 corresponds to steel)
        temp_add_back=np.zeros(rand_set_aside.shape) #(y,z,j) create a zero tensor for adding back later
        temp_add_back_mass=np.zeros((rand_set_aside.shape[0],1,rand_set_aside.shape[2])) #(y,1,j) create a tensor to record the mass of metal groups that are added back
        add_back_mass_cap=np.sum(rand_set_aside,axis=1,keepdims=True) #(y,1,j) create a mass cap tensor that limits the total mass (NOT individual AE mass) that can be added back to a given metal group
        #zero_tensor_mass_cap=np.zeros(add_back_mass_cap.shape) #(y,1,j) create a zero tensor with the same shape of mass cap
        np.random.shuffle(temp_order_all) #randomly shuffle the index, so steel (index=0) is not necessarily the first in the order array
        #convert array to list
        temp_order_all=temp_order_all.tolist()
        
        ##loop through randomly shuffled index of ALL metal groups:        
        for index1 in temp_order_all: 
            #if the CURRENT metal group does not have any set-aside mass (e.g., no Mg is used in engine), then continue to next iteration
            if np.sum(rand_set_aside[index1,:,:])==0: #this means no such metal group is ever used in ANY of the components (otherwise you should still do the following code)
                continue
            else:
                #randomly shuffle the index of set-aside mass of OTHER metal groups
                temp_order_other=deepcopy(temp_order_all)
                temp_order_other.remove(index1) #remove the index of CURRENT metal group first
                np.random.shuffle(temp_order_other) 
                #append the index of CURRENT metal to the end, in case when you have used up mass of all OTHER metal groups
                temp_order_other.append(index1)
                
                for index2 in temp_order_other:
                    if np.sum(temp_add_back[index1,:,:]) < np.sum(add_back_mass_cap[index1,:,:]): #as long as there is one component that needs more mass to be added back, do the following
                        #calculate the appropriate mass to add back first (1,j)
                        temp_to_add_mass=np.minimum(add_back_mass_cap[index1,:,:]-np.sum(temp_add_back[index1,:,:],axis=0,keepdims=True)
                                                             ,np.sum(rand_set_aside[index2,:,:],axis=0,keepdims=True)) #attention: axis is NOT 1, because you've lost the row (index1 or index2) dimension
                        temp_add_back_mass[index1,:,:]+=temp_to_add_mass
                        #then calculate the AE mass to add back (z,j)
                        temp_to_add_AE=np.multiply(temp_to_add_mass,rand_set_aside_conc[index2,:,:]) #(z,j)
                        temp_add_back[index1,:,:]+=temp_to_add_AE
                        #update corresponding AE mass in rand_set_aside[index2,:,:]
                        rand_set_aside[index2,:,:]-=temp_to_add_AE
                    else:
                        break # if the add_back value for CURRENT metal group is met, no need to loop through the rest of the OTHER metals
                        
                
        ##add back
        to_be_remelted+=temp_add_back

    #keep a copy of sorted scraps before remelting: used for "back-calculating" scraps that need not to be remelted (due to re-balancing)
    to_be_remelted_copy=deepcopy(to_be_remelted)
    
    """remelting (explicitly consider thermodynamic limits)"""
    ##initiate "IntAlloy_by_compnt" and "slag_loss_to_others"
    IntAlloy_by_compnt=np.zeros((row,col,channel))
    remelt_loss_to_others=np.zeros((row,col,channel))
    
    ##Calculate the resulting intermediate scrap alloy by metal group, and lost during remelting
    for j in range(to_be_remelted.shape[2]):
        IntAlloy_by_compnt[:,:,j]=np.multiply(to_be_remelted[:,:,j],TD_matrix)
        remelt_loss_to_others[:,:,j]=to_be_remelted[:,:,j]-IntAlloy_by_compnt[:,:,j]


    
    
    return IntAlloy_by_compnt,remelt_loss_to_others,to_be_remelted_copy




### Optimization-based algorithm for Blending

In [ ]:
def dilution_AddAE_rebal_combined(Int_Target_AE_alloc_by_compnt_ini,AE_matrix,TargetAlloy_mass_by_compnt,
                                  ingot_alloy_loss_matrix,alpha,num_iter,epsilon,total_utility_score_goal,alloy_group_cutoff):
    """this function is called inside the Blending_func when dilution is explicitly considered; this function uses 
        initial allocation of intermediate scrap alloys and AE spec to generate the mass of virgin bulk metal and
        AEs (collectively labeled as 'AE' in this function) needed, as well as the actual mass of intermediate scrap 
        alloys that are utilized
        
    **this function combines the "dilution", “AE addition” and “re-balancing” steps into one**
    **this function DOES NOT distinguish between 'bulk metal' and 'AEs', everything is labeled as 'AE'**
    **this function aims to minimize the use of virgin materials
    
    ## Input variables:
    -Int_Target_AE_alloc_by_compnt_ini: a 3d tensor with INITIAL offset of target alloys by intermediate scrap alloys(row),
        by AE mass (col),for each component OR car hulk (channel), in kg, (nc,z,j)
    -AE_matrix: target alloys (row) by alloying elements (col) matrix that shows the AE composition of each component, in %, (nc,z) 
    -TargetAlloy_mass_by_compnt:a 3D tensor that contains target alloy by metal (row), by AE composition (col), 
        for each component OR car hulk (channel), in total mass (kg), (nc,z,j)
    -ingot_alloy_loss_matrix: a matrix of alloys from secondary (row) and corresponding cumulative loss from 'ingot' stage to 'alloy' stage, (nc,1)
    -alpha: learning rate, scalar (dim=0)
    -num_iter: number of iterations before the gradient descent algorithm is terminated, scalar (dim=0)
    -epsilon: a small value to create margins of AE_spec%, in %
    -total_utility_score_goal: a small value to compare with total utility score of an iteration as the termination criteron , scalar (dim=0)
    -alloy_group_cutoff: a list contains tuple of (staring,ending) rows in the alloy-related matrix that belongs to a certain common metal group,
        len()= #of metal types
        
    ## Output variables:
    -output_Int_Target_mass_alloc_by_compnt_updated: a 3d tensor with UPDATED offset of target alloys by intermediate scrap alloys(row),
        by total mass (col=1),for each component OR car hulk (channel), in kg, (nc,1,j)
    -output_virgin_AE_mass_by_compnt: a 3d tensor with mass of virgin AEs (including bulk) (row=1) needed for blending
        by AE type (col),for each component OR car hulk (channel), in kg, (1,z,j)
    -output_virgin_alloy_needed_by_compnt:a 3d tensor that stores the mass (col=1) of each target alloy from virgin sources(row),
        for each component OR car hulk (channel), in total mass (nc,1,j) 
    -output_virgin_alloy_offset_by_compnt: a 3d tensor that stores mass (col=1) of each target alloy offset by IntAlloy+AE (row), 
        for each component OR car hulk (channel), in total mass (nc,1,j)

    """
    
    
    """Initialize variables"""
    dim_z=AE_matrix.shape[1]
    dim_j=TargetAlloy_mass_by_compnt.shape[2]
    
    ###expand dims
    ##expand AE_matrix to a 3d tensor (nc,z,1)
    #check if AE_matrix has already been extended to 3d outside the function
    if len(AE_matrix.shape)==3:
        AE_matrix_by_compnt=AE_matrix
    else:
        AE_matrix_by_compnt=np.expand_dims(AE_matrix,axis=2)
        AE_matrix_by_compnt=np.repeat(AE_matrix_by_compnt,dim_j,axis=2) #(nc,z,j)
  
    ##expand 'ingot-to-alloy' matrix to a 3d tensor (nc,1,1)
    #check if the 'ingot-to-alloy' matrix has already been extended to 3d outside the function
    if len(ingot_alloy_loss_matrix.shape)==3:
        ingot_alloy_loss_matrix_by_compnt=ingot_alloy_loss_matrix
    else:
        ingot_alloy_loss_matrix_by_compnt=np.expand_dims(ingot_alloy_loss_matrix,axis=2)
        ingot_alloy_loss_matrix_by_compnt=np.repeat(ingot_alloy_loss_matrix_by_compnt,dim_j,axis=2) #(nc,1,j)

   
    zero_tensor=np.zeros(TargetAlloy_mass_by_compnt.shape) #(nc,z,j)
    
    ###creat margin of AE_matrix for quicker convergence
    AE_matrix_by_compnt_margin_min=AE_matrix_by_compnt*(1-epsilon)
    AE_matrix_by_compnt_margin_max=AE_matrix_by_compnt*(1+epsilon)
    
    ###initial concentration of AE in intermediate scrap alloys
    #CAUTION: may contain 'nan's, as you may have 0.0/0.0 for certain AE in the IntAlloy
    AE_conc_IntAlloy_ini=np.divide(Int_Target_AE_alloc_by_compnt_ini,np.sum(Int_Target_AE_alloc_by_compnt_ini,axis=1,keepdims=True)) 
    #so, convert 'nan's to zeros (i.e., the mass of this IntAlloy should be zero)
    AE_conc_IntAlloy_ini=np.nan_to_num(AE_conc_IntAlloy_ini)
        
    ###initialize AE% of resulting (after blending) ingot (nc,z,j)
    AE_percent_i_j=AE_conc_IntAlloy_ini
    
    ###initialize beta
    beta=np.abs(AE_percent_i_j-AE_matrix_by_compnt)/AE_matrix_by_compnt

    ###update total mass of target INGOT(m_i_target) by considering the 'ingot-to-alloy' loss
    m_i_target=np.sum(np.multiply(TargetAlloy_mass_by_compnt,ingot_alloy_loss_matrix_by_compnt),axis=1,keepdims=True) #(nc,1,j)

    ###calculate total mass demand for AE, regardless of source (virgin or secondary)
    AE_mass_demand_regardless=np.multiply(AE_matrix_by_compnt,m_i_target) #(nc,z,j)

    ###initialize actual mass of IntAlloy allocation (IntA_i) tensor (nc,1,j)
    Int_Target_mass_alloc_by_compnt_ini=np.sum(Int_Target_AE_alloc_by_compnt_ini,axis=1,keepdims=True)
    Int_Target_mass_alloc_by_compnt_updated=Int_Target_mass_alloc_by_compnt_ini

    ###adjustment to the initil allocation of IntAlloy to avoid over alloc of AE
    ##identify the 'over_alloc_IntAlloy' (too much IntAlloy is allocated, as AE_i_j from IntAlloy > total demand)
    #create a copy of initial allocation by AE mass
    temp_AE_mass_secondary=np.multiply(AE_conc_IntAlloy_ini,Int_Target_mass_alloc_by_compnt_updated) #(nc,z,j)
    #create a filter for the difference between allocated AE mass and total demand of AE mass
    over_alloc_IntAlloy_filter=temp_AE_mass_secondary-AE_mass_demand_regardless
    
    #set the 'temp_AE_mass_secondary_diff' to 'over_alloc_IntAlloy_filter' to represent the difference
    temp_AE_mass_secondary_diff=over_alloc_IntAlloy_filter
    
    #create a pool of adjustment to make 
    pool_of_adjust=np.nan_to_num(temp_AE_mass_secondary_diff/AE_conc_IntAlloy_ini)
    #You will end up with'-1.79769313e+308', the largest possible negative value in python, for some cells because you are dividing by zero (0% AE)
    #   this should show '-inf', but after you apply 'np.nan_to_num', '-inf' becomes '-1.79769313e+308'
    #   such a gigantic negative value does not make sense (you probably need all the scrap ingots in the world in order to add this amount back to 'under-allocated' rows)

    #create a temporary storage for max of pool_of_adjust
    temp_adjust_max=np.max(pool_of_adjust,axis=1,keepdims=True) #(nc,1,j)
    temp_adjust_max[temp_adjust_max<-1.7e+308]=0.0 #set such a crazy value to zero

    """update the 'temp_adjust_max' to adjust the negative values of rows that are originally 'under-allocated'"""
    for cutoff_tuple in alloy_group_cutoff:
        temp_adjust_select=deepcopy(temp_adjust_max[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]) #store the rows of metal group, (k,1,j)
        #sum up the excessive mass (needed to be removed) and deficit mass (needs to be added)
        #if you directly use "temp_excessive=np.sum(temp_adjust_select[temp_adjust_select>0],axis=0,keepdims=True)", 
        #   you will have a shape of (1,), because the selection by boolean mask returns a one-dim array
        #also make copies of "temp_adjust_select", because the changes you are about to make, using boolean mask, will be 'inplace' (i.e., you don't want to mess up with the original data)
        temp_adjust_select_copy_for_excessive,temp_adjust_select_copy_for_deficit=deepcopy(temp_adjust_select),deepcopy(temp_adjust_select)
        #update the values in the respective arrays
        temp_adjust_select_copy_for_excessive[temp_adjust_select<0]=0.0 #set all negative values to zero for 'excessive' array
        temp_adjust_select_copy_for_deficit[temp_adjust_select>0]=0.0 #set all positive values to zero for 'deficit' array
        #because '1.79769313e+308' is the largest possible (absolute) value in python, when you have more than one of '-1.79769313e+308', you will get '-inf'
        #temp_adjust_select_copy_for_deficit[temp_adjust_select_copy_for_deficit<-1.7e+308]=0.0 #such a crazy large negative value is not meaningful in real life anyway (i.e., you won't be able to reassign this much scrap ingot)
        
        temp_excessive=np.sum(temp_adjust_select_copy_for_excessive,axis=0,keepdims=True) #(1,1,j)
        temp_deficit=np.sum(temp_adjust_select_copy_for_deficit,axis=0,keepdims=True) #(1,1,j)
        
        #determine the TOTAL mass to be added to rows where original assignment is “under-allocated”
        #multiply by -1 converts the mass to negative, so that it will lead to addition later in "Int_Target_mass_alloc_by_compnt_updated-=temp_adjust_max"
        temp_to_be_reassigned=np.minimum(temp_excessive,np.abs(temp_deficit))*-1 #(1,1,j)
        #attribute the temp_to_be_reassigned based on the ratio of rows that need additional scrap ingot
        temp_reassign_ratio=np.nan_to_num(np.divide(temp_adjust_select_copy_for_deficit,temp_deficit)) #'temp_adjust_select_copy_for_deficit' should only have negative values now
        
        #update 'temp_adjust_select': because the original negative values in 'temp_adjust_select' may be more than what's available from 'excessive mass' (that needs to be removed from other rows)
        #   so need to replace the original one with calculated one here
        temp_adjust_max[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:][temp_adjust_max[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]<0]=0.0 #set original negative values to zero first
        temp_adjust_max[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]+=np.multiply(temp_to_be_reassigned,temp_reassign_ratio) #add back the new negative values (the positive value rows should be affected)
                                                    
    
    #update the allocation of intermediate scrap alloys
    Int_Target_mass_alloc_by_compnt_updated-=temp_adjust_max
    #there may be very small negative value, as you did division so there may be rounding errors
    Int_Target_mass_alloc_by_compnt_updated[Int_Target_mass_alloc_by_compnt_updated<0.00000000001]=0.0
 
    
    ###initialize mass of target INGOT after offset (assume no AE addition is needed, i.e., 1:1 substitution)
    """caution: you may have after_offset alloy equal to total target demand, when no intermediate scrap alloy is used for offset
                which could cause "nan" or "inf" issue in utility function calculation below
    """
    m_i_after_offset=np.maximum(np.sum(zero_tensor,axis=1,keepdims=True),(m_i_target-Int_Target_mass_alloc_by_compnt_updated)) #(nc,1,j)
    
    ###accordingly, set initial mass of virgin AE (including both bulk and AE) needed to be added to zero
    """actually set to a small positive number to help converage;
            the value needs to be very small (e.g., 10E-7) otherwise you may have an artifact of less than 100% utilization ratio 
            for alloys that are easy to converge (i.e., AE% very close) and with a very small demand (e.g., 0.25 kg of Mg per vehicle)
    """
    #delta_AE_i_j=zero_tensor #(nc,z,j)
    delta_AE_i_j=np.ones(zero_tensor.shape)*0.0000001 #(nc,z,j)
    
    """reinitiate 'm_i_after_offset' to incorporate the 'delta_AE_i_j', such that both variables are from the same iteration"""
    m_i_after_offset-=np.sum(delta_AE_i_j,axis=1,keepdims=True)
    
    ###initialize utility score (make sure it's larger than the goal, otherwise 'while loop' won't work)
    utility_score_tensor=np.sum(zero_tensor,axis=1,keepdims=True)+total_utility_score_goal*2 #(nc,1,j) 
    average_utility_score=np.nanmean(utility_score_tensor,axis=(0,2)) #(1,)

    ###convert total utility score goal from a SCALAR to a Tensor
    total_utility_score_goal_tensor=np.ones(utility_score_tensor.shape)*total_utility_score_goal

    
    """gradient descent (for notation, refer to SI of the manuscript)"""
    
    ###initialize counter
    iter_counter=0
    
    while iter_counter<num_iter and np.any(utility_score_tensor>total_utility_score_goal_tensor):

        ###common term
        common_term_1=(np.multiply(AE_conc_IntAlloy_ini,Int_Target_mass_alloc_by_compnt_updated)+delta_AE_i_j)/(m_i_target-m_i_after_offset)-AE_matrix_by_compnt #(nc,z,j)
        #if m_i_target-m_i_after_offset=0, then common_term1 will have 'inf' because the numerator is not zero
        #so set 'inf' to zero
        common_term_1[np.isinf(common_term_1)]=0
        common_term_2=1.0/(dim_z*((m_i_target-m_i_after_offset))) #(nc,1,j)
        #similarly, handle 'inf' for common_term_2
        common_term_2[np.isinf(common_term_2)]=0
        #you may have initial value for virgin_alloy_still_needed that is equal to target, when no scrap alloy is used for offset
        common_term_0=1.0/(2*dim_z)

        ###utility score
        square_term=np.square(common_term_1)
        
        """leave the 'nan' as is in 'utility_score_tensor' and calculate an average score to use for While loop"""
        utility_score_tensor=common_term_0*np.sum(square_term,axis=1,keepdims=True) #(nc,1,j)
        #calculate an average score to use for While loop: average over axis=0 (nc), then axis=2 (j)
        average_utility_score=np.nanmean(utility_score_tensor,axis=(0,2)) #(1,)
                
        ###calculate AE% of resulting ingot
        AE_percent_i_j=(np.multiply(AE_conc_IntAlloy_ini,Int_Target_mass_alloc_by_compnt_updated)+delta_AE_i_j)/(m_i_target-m_i_after_offset)
        #if m_i_target-m_i_after_offset=0, then AE_percent_i_j will have 'inf' because the numerator is not zero
        #AE_percent_i_j[np.isinf(AE_percent_i_j)]=0
        #print "AE% after blending for [last alloy (AZ91),:,first compnt] : ", AE_percent_i_j[-1,:,0],"\n"
        
        ###derivative wrt additional vrigin AE (dU/dTheta_i_j), (nc,1,j)
        #CAUTION: this derivative is the same for ALL AE (j), so you will need to have AE-specific adjustment (beta)
        dTheta_i_j=common_term_2*np.sum(np.abs(common_term_1),axis=1,keepdims=True)
        
        ###derive beta
        #if the AE% is already within the margin, no need to update Thetas, hence beta=0 (so set all elements to zero first)
        beta=np.zeros(beta.shape)
        #create a filter to update elements where AE_percent_i_j is NOT within the margin
        filter_outside_margin_1=AE_percent_i_j<AE_matrix_by_compnt_margin_min
        filter_outside_margin_2=AE_percent_i_j>AE_matrix_by_compnt_margin_max
        filter_outside_margin=filter_outside_margin_1 ^ filter_outside_margin_2        
                        
        beta[filter_outside_margin]=np.abs(AE_percent_i_j[filter_outside_margin]-AE_matrix_by_compnt[filter_outside_margin])/AE_matrix_by_compnt[filter_outside_margin]
        beta[np.where(np.isinf(beta))]=1.0
        
        beta*=alpha
        
        ###update delta_AE
        delta_AE_i_j+=np.multiply(beta,dTheta_i_j)
        
        #if there is "nan" in delta_AE_i_j, then it means "m_i_target-m_i_after_offset=0", i.e., after_offset virgin alloy
        #is equal to target demand and no scrap alloy is used for offset-->thus no any delta_AE (j) of this alloy (i) is needed to be added
        delta_AE_i_j=np.nan_to_num(delta_AE_i_j)
        
        #calcualte virgin alloy still needed
        m_i_after_offset=m_i_target-Int_Target_mass_alloc_by_compnt_updated-np.sum(delta_AE_i_j,axis=1,keepdims=True)
        
        
        """to re-assign the removed scrap ingot to other rows"""
        ##Make a copy of ‘m_i_after_offset’ and set all positive values to zero, all negative values to positive
        excess_scrap_ingot_available_for_removal=m_i_after_offset.copy() #this creates a deep copy of the array
        excess_scrap_ingot_available_for_removal[excess_scrap_ingot_available_for_removal>0]=0.0 #set all positive values (where virgin alloys are still needed after blending) to zero 
        excess_scrap_ingot_available_for_removal*=-1 #set all negative values to positive (this becomes the postive amount of scrap ingot that can be reassigned)
        
        ##update the 'Int_Target_mass_alloc_by_compnt_updated', the relevant rows are those correspond to negative values in "m_i_after_offset"
        #make a copy of 'Int_Target_mass_alloc_by_compnt_updated'
        copy_Int_Target_mass_alloc_by_compnt_updated=Int_Target_mass_alloc_by_compnt_updated.copy()
        Int_Target_mass_alloc_by_compnt_updated[m_i_after_offset<0]-=excess_scrap_ingot_available_for_removal[m_i_after_offset<0]
        #check if any of the corresponding rows in 'Int_Target_mass_alloc_by_compnt_updated' becomes negative or zero
        #  if so, it means too much 'delta_AE' is added (i.e., the quality of scrap ingot is too low for recycling for this particular row of targeta alloy)
        #so, first adjust delta_AE to set it to zero when scrap ingot assignment is negative or zero
        negative_scrap_assign_index=np.where(Int_Target_mass_alloc_by_compnt_updated<=0) #(nc,1,j)
        delta_AE_i_j[negative_scrap_assign_index[0],:,negative_scrap_assign_index[2]]=0.0 #as the second dim of crazy_index is 1, need manually set all second dim (z) of delta_AE_i_j to zero
        #then, set negative or zero assignment of scrap ingot to zero
        Int_Target_mass_alloc_by_compnt_updated[Int_Target_mass_alloc_by_compnt_updated<=0]=0.0
        #Update ‘m_i_after_offset’  accordingly
        m_i_after_offset=m_i_target-Int_Target_mass_alloc_by_compnt_updated-np.sum(delta_AE_i_j,axis=1,keepdims=True)
        #don't forget to update "excess_scrap_ingot_available_for_removal": 
        #  if the mass to be removed is more than originally assigned (due to extremely high AE addition),
        #  set the removal equal to the orignial assignment
        more_than_original_index=np.where(excess_scrap_ingot_available_for_removal>copy_Int_Target_mass_alloc_by_compnt_updated)
        excess_scrap_ingot_available_for_removal[more_than_original_index]=copy_Int_Target_mass_alloc_by_compnt_updated[more_than_original_index]
            
        ##update the 'Int_Target_mass_alloc_by_compnt_updated' again, to re-assign the removed scrap ingots to those rows where virgin alloy is still needed
        for cutoff_tuple in alloy_group_cutoff:
            temp_supply_for_reassign_select=deepcopy(excess_scrap_ingot_available_for_removal[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]) #store a copy of the rows of metal group that can provide removed scrap ingots, (k,1,j)
            temp_sink_for_reassign_select=deepcopy(m_i_after_offset[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]) #'m_i_after_offset' contains the rows where virgin alloys are still used (i.e., may potentially take up removed scrap ingots, (k,1,j)
                                                                                                            
            #sum up the excessive mass (available for reassignment) and sink mass (can potentially take these excessive mass)
            temp_supply_total=np.sum(temp_supply_for_reassign_select,axis=0,keepdims=True) #(1,1,j)
            temp_sink_total=np.sum(temp_sink_for_reassign_select,axis=0,keepdims=True) #(1,1,j)
        
            #determine the TOTAL mass to be added to rows where virgin alloys are still needed after blending
            temp_to_be_reassigned_in_loop=np.minimum(temp_supply_total,temp_sink_total) #(1,1,j)

            #attribute the temp_to_be_reassigned based on the ratio of rows that need additional scrap ingot
            temp_reassign_ratio_in_loop=np.nan_to_num(np.divide(temp_sink_for_reassign_select,temp_sink_total)) #(k,1,j)
            
            #update 'Int_Target_mass_alloc_by_compnt_updated' with additional scrap ingot use
            Int_Target_mass_alloc_by_compnt_updated[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]+=np.multiply(temp_to_be_reassigned_in_loop,temp_reassign_ratio_in_loop) #add back the new negative values (the positive value rows should be affected)
            #Update ‘m_i_after_offset’  accordingly
            m_i_after_offset=m_i_target-Int_Target_mass_alloc_by_compnt_updated-np.sum(delta_AE_i_j,axis=1,keepdims=True)
   
        
        ##check mass balance
        """This function may leave some very small discrepancy in mass balance (e.g., 10E-10)"""
        mass_balance=m_i_after_offset+Int_Target_mass_alloc_by_compnt_updated+np.sum(delta_AE_i_j,axis=1,keepdims=True)-m_i_target

        
        ###move to next iteration
        iter_counter+=1
    
    """check AE mass balance (based on the latest iteration)"""
    AE_mass_balance=np.multiply((Int_Target_mass_alloc_by_compnt_updated+np.sum(delta_AE_i_j,axis=1,keepdims=True)),AE_percent_i_j)+np.multiply(m_i_after_offset,AE_matrix_by_compnt)-np.multiply(m_i_target,AE_matrix_by_compnt) #(nc,z,j)
    
    temp_AE_balance_for_print=AE_mass_balance.copy()
    temp_AE_balance_for_print[np.isinf(AE_mass_balance)]=0.0
    print ("total mass imbalance as a % of total material demand: ", np.sum(abs(np.nan_to_num(temp_AE_balance_for_print)))/np.sum(np.nan_to_num(m_i_target))*100,"%","\n")

    #output the AE mass balance result to csv
    np.savetxt("AE mass balance (should be all zeros)_CSV.csv", AE_mass_balance[:,:,0], delimiter=",")
    
    ##check iteration numbers and average utility scores
    print ("the End, iter_counter is: ", iter_counter,"\n")  
    print ("final average utility score is: ", average_utility_score, "\n")
    
    
    """create output variables and their values"""
    #left_over IntAlloy will be calculated outside this function
    output_Int_Target_mass_alloc_by_compnt_updated=Int_Target_mass_alloc_by_compnt_updated #(nc,1,j)
    output_virgin_AE_mass_by_compnt=delta_AE_i_j #(nc,z,j)
    output_virgin_alloy_needed_by_compnt=np.divide(m_i_after_offset,ingot_alloy_loss_matrix_by_compnt) #(nc,1,j), to convert from 'ingot' needed to 'final alloy' needed
    output_virgin_alloy_offset_by_compnt=np.sum(TargetAlloy_mass_by_compnt,axis=1,keepdims=True)-output_virgin_alloy_needed_by_compnt #(nc,1,j)
      
    """output (1) AE% of resulting ingots, (2) difference between resulting ingots and spec, (3) AE_matrix_by_compnt"""
    #[caution]this always output the results for the last vehicle type (i.e., EV light)
    np.savetxt("AE%_of_resulting_ingots_CSV.csv", AE_percent_i_j[:,:,0], delimiter=",")
    np.savetxt('difference_between_blended_and_spec_CSV.csv',AE_percent_i_j[:,:,0]-AE_matrix_by_compnt[:,:,0],delimiter=",")
    
    ##also print out the AE_matrix_by_compnt (should be exactly/very close to AE matrix that is provided exogenously)
    #[caution]this always output the results for the last vehicle type (i.e., EV light)
    np.savetxt('AE_spec_from_blending_step.csv',AE_matrix_by_compnt[:,:,0],delimiter=",")
    
    return output_Int_Target_mass_alloc_by_compnt_updated,output_virgin_AE_mass_by_compnt,output_virgin_alloy_needed_by_compnt,output_virgin_alloy_offset_by_compnt    

### Blending function 

In [ ]:
def Blending(IntAlloy_by_compnt,TargetAlloy_mass_by_compnt,AE_matrix,alloy_group_cutoff,ingot_alloy_loss_matrix,
             alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,
             IntAlloy_mass_alloc='Euclidean distance',dilution_explicit=True,one_very_small_value=0.00001,**kwargs):
    """This function calculates the virgin material use for blending (with intermediate scrap alloys) for different components
    ## Input variables
    -IntAlloy_by_compnt: a 3D tensor that contains intermediate scrap alloy by metal group (row), by AE composition (col), 
        for each component OR car hulk (channel), in total mass (kg), (y,z,j)
        *if "Cross-Mixing", this input should be partial of total (based on the ratio of vehicle demand by type)
    -AE_matrix: alloys (row) by alloying elements (col) matrix that shows the AE composition of each component, in %, (n,z) 
    -TargetAlloy_mass_by_compnt:a 3D tensor that contains target alloy by metal (row), by AE composition (col), 
        for each component OR car hulk (channel), in total mass (kg), (n,z,j)
    -alloy_group_cutoff: a list contains tuple of (staring,ending) rows in the alloy-related matrix that belongs to a certain common metal group,
        len()= #of metal types
    -ingot_alloy_loss_matrix: a matrix of alloys from secondary (row) and corresponding cumulative loss from 'ingot' stage to 'alloy' stage, (n,1)
    -IntAlloy_mass_alloc: method to allocate intermediate scrap alloy for blending for target alloy, in text
        "cosine similarity": allocate the mass of intermediate scrap alloy (of a certain metal group) based on the order of 
            how "close" the composition is between the scrap alloy and target alloy
        Or
        "distribution by target ratio": allocate the mass of intermediate scrap alloy (of a certain metal group) 
            based on the (total mass) ratio of target alloys
        Or
        "Euclidean distance": allocate the mass of intermediate scrap alloy (of a certain metal group) 
            based on the Euclidean distance of AE concentrations between the scrap and target alloys
    -dilution_explicit: a boolean variable to indicate whether "dilution (quality loss)" is explicitly modeled
    -closed_loop_recycl_rate_list: a list containing the metal group-lvl closed-loop recycling rate factors, in %
    ***below four are added for the optimization-based dilution function***
    -alpha: learning rate, scalar (dim=0)
    -num_iter: number of iterations before the gradient descent algorithm is terminated, scalar (dim=0)
    -epsilon: a small value to create margins of AE_spec%, in %
    -total_utility_score_goal: a small value to compare with total utility score of an iteration as the termination criteron , scalar (dim=0)
    ***above four are added for the optimization-based dilution function***
    -keyword arguments:
        weight_vector: this gives weight for each AE element (z,) during mass allocation of intermediate scrap alloy, e.g., when a certain AE is given a high weight
            more of the intermediate scrap alloy may be allocated to produce target alloy between which the difference in AE mass is the smallest
            (i.e., less virgin AE of high weight is needed)
    ## Output variables
    -virgin_bulk_metal_mass_by_compnt: a 3D tensor containing virgin bulk metals (row), by total mass (col), for each component OR car hulk (channel), in total mass (kg), (y,1,j)
    -virgin_AE_mass_by_compnt: a 3D tensor containing (1 row of) virgin AEs needed (col), for each component OR car hulk (channel), in total mass (kg), (1,z,j)
    -Int_Target_mass_alloc_by_compnt: a 3D tensor that stores the mass of intermediate scrap alloy (col=1) allocated to each target alloy (row),
        for each component OR car hulk (channel), in total mass (n,1,j)
    -Int_Target_AE_alloc_by_compnt: a 3D tensor that stores the mass of AEs (including bulk) that constitute intermediate scrap alloy (col=z),
        allocated to each target alloy (row),for each component OR car hulk (channel), in total mass (n,z,j)
    -virgin_alloy_needed_by_compnt: a 3D tensor that stores the mass (col=1) of each target alloy from virgin sources(row),
        for each component OR car hulk (channel), in total mass (n,1,j)
    -IntAlloy_leftover_by_compnt: a 3D tensor that stores the AE mass (col=z) of each intermediate scrap alloy that is not used up (row),
        for each component OR car hulk (channel), in total mass (y,z,j)
    -virgin_alloy_offset_by_compnt: a 3D tensor that stores the mass (col=1) of each target alloy from secondry sources (row),
        for each component OR car hulk (channel), in total mass (n,1,j)
    -IntAlloy_mass_alloc_by_compnt: a 3D tensor that stores the mass (col=1) of each intermediate scrap alloy that is used for blendng(row),
        for each component OR car hulk (channel), in total mass (y,1,j)
    """
        
    
    """(Initial) intermediate scrap alloy ALLOCATION for target alloy production"""
    ##Initiation
    virgin_bulk_metal_mass_by_compnt=np.zeros((IntAlloy_by_compnt.shape[0],1,IntAlloy_by_compnt.shape[2])) #(y,1,j)
    virgin_AE_mass_by_compnt=np.zeros((1,IntAlloy_by_compnt.shape[1],IntAlloy_by_compnt.shape[2])) #(1,z,j)
    IntAlloy_leftover_by_compnt=np.zeros((IntAlloy_by_compnt.shape[0],IntAlloy_by_compnt.shape[1],IntAlloy_by_compnt.shape[2])) #(y,z,j)
    Int_Target_mass_alloc_by_compnt=np.zeros((TargetAlloy_mass_by_compnt.shape[0],1,TargetAlloy_mass_by_compnt.shape[2])) #(n,1,j)
    Int_Target_AE_alloc_by_compnt=np.zeros((TargetAlloy_mass_by_compnt.shape[0],TargetAlloy_mass_by_compnt.shape[1],TargetAlloy_mass_by_compnt.shape[2])) #(n,z,j)
    virgin_alloy_needed_by_compnt=np.sum(TargetAlloy_mass_by_compnt,axis=1,keepdims=True) #initialize the virgin alloy need to total alloy need (assume no scrap recycling yet)
    Int_Target_AE_percent_ini=np.zeros(Int_Target_AE_alloc_by_compnt.shape) #AE% in intially allocated scrap alloys (n,z,j)

    """adjust target alloy demand to 'target ingot demand' """
    #expand the dimension of 'ingot_alloy_loss_matrix' from (nc,1) to (nc,1,1)
    ingot_alloy_loss_matrix=ingot_alloy_loss_matrix.as_matrix()    
    ingot_alloy_loss_matrix=np.expand_dims(ingot_alloy_loss_matrix,axis=2) #expand the dimension to 3d, should be (nc,1,1) now
    #make sure "virgin_alloy_needed_by_compnt" is not affected
    TargetAlloy_mass_by_compnt=np.multiply(TargetAlloy_mass_by_compnt,ingot_alloy_loss_matrix)

    ###calculate the AE% in intermediate scrap alloys and target alloys, as they will be used by "distance-based" (initial) allocation and Dilution step
    ##Target alloy specs first (caution: as not all target alloys will be used in each component)
    temp_target_mass_adjust=np.sum(TargetAlloy_mass_by_compnt, axis=2,keepdims=True) #(n,z,1) collapse along component dimension to make sure each alloy has a non-zero value
                                                                                        #make sure each target alloy is used by at least one component
    

    ##assign the matrix (n,z) first
    TargetAlloy_AE_percent_spec=AE_matrix
    ##then expand to 3D
    TargetAlloy_AE_percent_spec=np.repeat(TargetAlloy_AE_percent_spec[:,:,np.newaxis],Int_Target_AE_percent_ini.shape[2],axis=2) #(n,z,j)
           
    ##Intermediate scrap alloy
    #calculate the AE% by metal group first    
    Int_Target_AE_percent_ini_by_metal_group=IntAlloy_by_compnt/np.sum(IntAlloy_by_compnt,axis=1,keepdims=True) #(y,z,j)
        
    #then expand to each target alloy in that metal group
    for index, cutoff_tuple in enumerate(alloy_group_cutoff):
        Int_Target_AE_percent_ini[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]=Int_Target_AE_percent_ini_by_metal_group[index,:,:]
    
    
    ###'distribution by target ratio' method is less complicated, as it allocates the mass all at once, 
    if IntAlloy_mass_alloc=='distribution by target ratio' or dilution_explicit==False:
        ##create a 3D tensor to normalize the target alloy (total mass) ratio for all components
        for index,cutoff_tuple in enumerate(alloy_group_cutoff):
            temp_target_ratio=np.sum(TargetAlloy_mass_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:],axis=1,keepdims=True) #(k,1,j), k rows are (within a metal group) preserved
            target_alloy_ratio=temp_target_ratio/np.sum(temp_target_ratio, axis=0,keepdims=True) #(k,1,j)
            ##(inital) allocate mass of intermeidate scrap alloy, take ONE row from y rows and spread by multiply with "target_alloy_ratio"
            #np.einsum here is equal to element-wise multiplication between the two arrays (with broadcasting), because axis "z" is retained
            Int_Target_mass_alloc_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]=np.einsum('lzj,kzj->kzj',np.sum(IntAlloy_by_compnt[index:index+1,:,:],axis=1,keepdims=True),target_alloy_ratio) #(k of n,(z=)1,j; l=1)    
            #also record the allocation in terms of AE mass
            Int_Target_AE_alloc_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]=np.einsum('lzj,kzj->kzj',IntAlloy_by_compnt[index:index+1,:,:],target_alloy_ratio) #(k of n,z(!=1 for IntAlloy, =1 for ratio),j; l=1) 

        Int_Target_mass_alloc_by_compnt=np.nan_to_num(Int_Target_mass_alloc_by_compnt)
        Int_Target_AE_alloc_by_compnt=np.nan_to_num(Int_Target_AE_alloc_by_compnt)

    ##After the lines above, (initial) allocation of mass of intermediate scrap alloys of ALL metal groups and ALL components are assigned##
  

    ###Distance-based methods share the same data pre-processing steps###
    else:
        #initiate variables
        Int_Target_dict=collections.defaultdict(list)
        
        try:
            weight_vector=kwargs['weight_vector'] #if there is a keyword argument input for weight matri, use it; otherwise, use default
        except:
            weight_vector=np.ones((IntAlloy_by_compnt.shape[1])) #default setting: all ones for z AEs in 1D array (z,)

        #sum up the mass of AEs to get the total mass of intermeidate alloys
        IntAlloy_total_mass_by_compnt=np.sum(IntAlloy_by_compnt,axis=1,keepdims=True) #(y,1,j)
            
        for compnt_index in range(TargetAlloy_mass_by_compnt.shape[2]): #loop through each component channel
                                                                        #can't carry the jth dimension in the calculation as cosine takes in 1D array only
            for index,cutoff_tuple in enumerate(alloy_group_cutoff): #cutoff for metal groups never change
                #update the Int_Target_dict every time you switch to another component channel
                Int_Target_dict[index]=TargetAlloy_mass_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,compnt_index] #the selected "portion" of "TargetAlloy_mass_by_compnt" has the dimension of (k,z,)
                                                                                                                    #total number of "index" is equal to # of metal groups (y)
            ##calculate similarity and allocate mass for each metal group   
            for IntAlloy_index,target_alloys in Int_Target_dict.items(): #number of rows of IntAlloy is the same as number of rows in cutoff tuple dict

                similarity_list=[] #initiate the similarity score as an empty list
                ranked_alloy_list=[]#for ranking the target alloys for a given metal group
                
                ##get the intermediate scrap alloy that will be used for comparison and allocation
                #get a total mass that will be REDUCED during each iteration of allocation
                temp_IntAlloy_total_mass=IntAlloy_total_mass_by_compnt[IntAlloy_index,:,compnt_index] #dimension=(1,)
                #create a copy of total mass that will NOT change during the allocation process
                temp_IntAlloy_total_mass_locked=IntAlloy_total_mass_by_compnt[IntAlloy_index,:,compnt_index] #dimension=(1,)
                temp_IntAlloy=np.squeeze(IntAlloy_by_compnt[IntAlloy_index,:,compnt_index]) #(z,)
                #find the corresponding scrap alloy concentration
                temp_IntAlloy_conc=Int_Target_AE_percent_ini_by_metal_group[IntAlloy_index,:,compnt_index] #(z,)
        
                """check for 'nan': skip this run if 'temp_IntAlloy_conc' contains 'nan' """
                if np.any(np.isnan(temp_IntAlloy_conc)):
                    #print "an 'nan' is found!"
                    #print temp_IntAlloy_conc
                    continue
                """
                by 'continue', you are skipping the 'zero mass' scrap alloys and hence avoid comparison of AE composition with target alloys (below)
                As this comparison is a 'one (intermediate scrap alloy) to many (target alloys)' problem, skipping one intermediate scrap alloy
                will avoid the comparision for all relevant target alloys, which is reasonable as you won't have scrap alloy to make any for these target alloys
                """
            
                #loop through targe alloys (k,z) of a given metal group , similarity=1-cosine distance
                for alloy in target_alloys: #by default takes each row (z,) of a tensor (k,z,)
                    temp_target=np.squeeze(alloy) #just as a precaution to make sure dimension =(z,)
                    #create a concentration version of temp_target
                    #caution: to avoid the situation where the concentrations are very similar between scrap and target
                    #but no demand of such target of a given component (i.e., allocatin should be zero)
                    if np.sum(temp_target)!=0: #this also helps to avoid the "division by zero" issue
                        temp_target_conc=temp_target/np.sum(temp_target,keepdims=True) #(z,)

                        ##calculate the similarity score and use it to rank the target alloy (array of mass) accordingly
                        ##placing specific method check here can be redundant, as within each component, you have to check it once, NOT GOOD##

                        ###'cosine similarity' method is more complicated, as each time a closest target alloy is identified,
                        ### then after the corresponding mass is allocated to meet the demand of that target alloy,
                        ### then the next closest target alloy is identified   
                        if IntAlloy_mass_alloc=='cosine similarity':
                            similarity_list.append(1-cosine(temp_IntAlloy_conc,temp_target_conc,w=weight_vector))
                        ###'Euclidean distance' method 
                        elif IntAlloy_mass_alloc=='Euclidean distance':
                            #need to increase the dimension to a 2D-array for Euclidean distance...
                            temp_shape=temp_target_conc.shape[0] #DO use temp_target_conc.shape[0] because it gets new input (z,) everytime
                            temp_IntAlloy_conc=temp_IntAlloy_conc.reshape((1,temp_shape)) #(1,z); this is actually redundant, as after 1st iteration, the dimension is already (1,z)
                            temp_target_conc=temp_target_conc.reshape((1,temp_shape)) #(1,z)
                            #as the target alloys will re-ordered by be similarity score (the higher the better)
                            #this means for Euclidean distance, the larger distance needs to transform to a lower score;
                            #therefore, take negative of the Euclidean distance as similarity score
                            similarity_list.append(-1*cdist(temp_IntAlloy_conc,temp_target_conc,w=weight_vector).tolist()[0][0]) #flatten the list, just as an precaution
                    else:
                        similarity_list.append(-99999) #assign very large negative value to put the "no-demand" alloys to the end of the list
                    #add results to the lists first (matching the index between 'similarity' and 'alloy' lists)
                    ranked_alloy_list.append(np.sum(alloy)) #each 'alloy' from (z,) to (1,)
            
                #then get the sorted index of the similartiy_list (for similarity value sorted from large to small)
                temp_new_order=np.argsort(similarity_list).tolist()
                temp_new_order.reverse() #this reverse the order of sorted index, such that higher similarity value ranks higher
                                        #as list.reverse() modifies the list 'inplace' and returns None, if you use chain command,
                                        #'temp_new_order' will be assigned None (returned by the last command 'list.reverse()')
                
                ##allocate the mass to target alloy (of a given metal group)                
                ##(linear) method
                #prepare a list of index that corresponds to the row numbers in the entire "Int_Target_mass_alloc_by_compnt" tensor 
                #    (should have the same dimension as other 3D tensors such as "virgin_alloy_needed_by_compnt")
                cutoff_tuple=alloy_group_cutoff[IntAlloy_index] #get the cutoff_tuple corresponding to this metal group
                global_row_index_of_k=[index for index in range(cutoff_tuple[0],cutoff_tuple[1]+1)] #means the global row index representation of the index of k rows for this loop
                                
                for sorted_index in temp_new_order: #start from the most similar alloy
                    if temp_IntAlloy_total_mass>0: #check if there is still mass left for making this specific target alloy
                        #allocate the mass that is equivalent or less than the demand of target alloy
                        temp_alloc=min(temp_IntAlloy_total_mass,ranked_alloy_list[sorted_index]) #make sure the mass in ranked_alloy_list[sorted_index] refers to the same row in "Int_Target_mass_alloc_by_compnt" tensor 
                        Int_Target_mass_alloc_by_compnt[global_row_index_of_k[sorted_index],:,compnt_index]=temp_alloc
                        #also record the allocation in terms of AE mass
                        Int_Target_AE_alloc_by_compnt[global_row_index_of_k[sorted_index],:,compnt_index]=(temp_alloc/temp_IntAlloy_total_mass_locked)*temp_IntAlloy
                        #reduce the allocated mass from total mass of intermediate scrap alloy
                        temp_IntAlloy_total_mass=max(0,temp_IntAlloy_total_mass-temp_alloc) 
                    else:
                        break
                
            ##After the line above, (initial) allocation of mass of intermediate scrap alloys of ALL metal groups and ALL components are assigned##
    
    
    """For explicitly considering DILUTION (see SI of the manuscript for details of the analytical solution)"""
    if dilution_explicit:     

        """generate virgin AE added, IntAlloy allocated for offset and virgin alloy still needed after offset use new optimization-based function"""
        output_Int_Target_mass_alloc_by_compnt_updated,output_virgin_AE_mass_by_compnt,output_virgin_alloy_needed_by_compnt,output_virgin_alloy_offset_by_compnt=dilution_AddAE_rebal_combined(Int_Target_AE_alloc_by_compnt,AE_matrix,
                                                                                                                                                            TargetAlloy_mass_by_compnt,ingot_alloy_loss_matrix,alpha,num_iter,epsilon,total_utility_score_goal,alloy_group_cutoff)

        
         
        """Update outputs (virgin bulk metals, AEs, alloys, leftover mass of intermediate scrap alloys)"""
        ##update leftover intermediate scrap alloy (y,z,j)
        for index, cutoff_tuple in enumerate(alloy_group_cutoff):
        #if scrap alloy is more than needed for blending, there will be no need for additional virgin alloy, but some leftover scrap or vice versa
            #use "index:index+1" to avoid the loss of a dimension
            IntAlloy_leftover_by_compnt[index,:,:]=IntAlloy_by_compnt[index:index+1,:,:]-np.sum(np.nan_to_num(np.multiply(output_Int_Target_mass_alloc_by_compnt_updated[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:], Int_Target_AE_percent_ini[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:])),axis=0,keepdims=True)
        #create a output tensor for IntAlloy_alloc_mass (y,1,j)
        IntAlloy_mass_alloc_by_compnt=np.sum(IntAlloy_by_compnt-IntAlloy_leftover_by_compnt,axis=1,keepdims=True)
        
        ##update the need of virgin AEs (1,z,j)
        virgin_AE_mass_by_compnt=np.sum(output_virgin_AE_mass_by_compnt,axis=0,keepdims=True)
        
        ##update the virgin alloy offset
        virgin_alloy_offset_by_compnt=output_virgin_alloy_offset_by_compnt
        
        ##update the need of virgin alloys (n,1,j)
        virgin_alloy_needed_by_compnt-=virgin_alloy_offset_by_compnt #alternatively you can also directly use the output from the new function

        ##add the 'AE' (actually bulk+AE) mass that consititutes virgin alloys (accounting for 'ingot-->alloy' loss)
        #to be used for calculating energy consumption associated with primary metal and AE demand
        #is double-counting of mass (as this AE mass is larger than virgin alloy demand)
        temp_AE_comp_tensor=np.expand_dims(AE_matrix,axis=2) #from (n,z) to (n,z,1)
        temp_AE_of_virgin_alloy=np.multiply(np.multiply(virgin_alloy_needed_by_compnt,temp_AE_comp_tensor),ingot_alloy_loss_matrix) #(n,z,j)
        #update total AE demand
        temp_AE_of_virgin_alloy=np.sum(temp_AE_of_virgin_alloy,axis=0,keepdims=True) #calculate total mass of AE by collapsing the along the alloys axis
        virgin_AE_mass_by_compnt+=temp_AE_of_virgin_alloy
        
        ##update Int_Target_mass_alloc_by_compnt
        Int_Target_mass_alloc_by_compnt=output_Int_Target_mass_alloc_by_compnt_updated
        

    else:
        """Update outputs (virgin alloys,leftover mass of intermediate scrap alloys), if NO DILUTION IS CONSIDERED"""
        ###since no dilution, there will be no demand for virgin bulk metal and AEs (i.e., disregard the quality loss from AE% difference) ###

        ##update the leftover intermediate scrap alloy (y,z,j)
        for index, cutoff_tuple in enumerate(alloy_group_cutoff):
            #update the value of corresponding rows in "Int_Target_AE_alloc_by_compnt"
            Int_Target_AE_alloc_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:]*=closed_loop_recycl_rate_list[index]
            IntAlloy_leftover_by_compnt[index,:,:]=IntAlloy_by_compnt[index:index+1,:,:]-np.sum(Int_Target_AE_alloc_by_compnt[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:], axis=0,keepdims=True)        
        #update 'Int_Target_mass_alloc_by_compnt' accordingly
        Int_Target_mass_alloc_by_compnt=np.sum(Int_Target_AE_alloc_by_compnt,axis=1,keepdims=True)
        #create a output tensor for IntAlloy_alloc_mass (y,1,j)
        IntAlloy_mass_alloc_by_compnt=np.sum(IntAlloy_by_compnt-IntAlloy_leftover_by_compnt,axis=1,keepdims=True)
        
        ##update the virgin alloy offset
        virgin_alloy_offset_by_compnt=np.divide(Int_Target_mass_alloc_by_compnt,ingot_alloy_loss_matrix)

        ##update the need of virgin alloys (n,1,j)
        virgin_alloy_needed_by_compnt-=virgin_alloy_offset_by_compnt
        
        ##add the 'AE' (actually bulk+AE) mass that consititutes virgin alloys (accounting for 'ingot-->alloy' loss)
        #to be used for calculating energy consumption associated with primary metal and AE demand
        #is double-counting of mass (as this AE mass is larger than virgin alloy demand)
        temp_AE_comp_tensor=np.expand_dims(AE_matrix,axis=2) #from (n,z) to (n,z,1)
        temp_AE_of_virgin_alloy=np.multiply(np.multiply(virgin_alloy_needed_by_compnt,temp_AE_comp_tensor),ingot_alloy_loss_matrix) #(n,z,j)
        #update total AE demand
        temp_AE_of_virgin_alloy=np.sum(temp_AE_of_virgin_alloy,axis=0,keepdims=True) #calculate total mass of AE by collapsing the along the alloys axis
        virgin_AE_mass_by_compnt+=temp_AE_of_virgin_alloy  

    
    return IntAlloy_mass_alloc_by_compnt,virgin_alloy_offset_by_compnt, virgin_alloy_needed_by_compnt,virgin_AE_mass_by_compnt,IntAlloy_leftover_by_compnt,Int_Target_mass_alloc_by_compnt,Int_Target_AE_alloc_by_compnt

 



### A general function for parsing data for Primary, Manu and EoL

In [ ]:
def metal_data_parsing(data_sheet_dict):
    """This module does data parsing for general purpose
    ## Input variables:
    -data_sheet_dict: a dict contains the data sheets (sheet object) that are used for parsing, in the format of {"sheet_name": sheet}
    ## Output variables:
    -data_parsed_dict: a dict contains the parsed data (in dict), in the format of {"sheet name": {metal name processed,process,unit":[min,max]}}
    -data_energy_actions_set_dict: a dict that contains set of actions for metal processing for each data sheet, in the format of {"sheet name": (process)}
    """
    #a temp dictionary to store the parsed data for each data sheet
    data_parsed_temp=collections.defaultdict(list)
    #get a SET of actions
    data_energy_actions_set=set()
    #an overall dictionary to store all parsed data for all data sheets
    data_parsed_dict=collections.defaultdict()
    #an overall dictionary to store all unique activities for each data sheet
    data_energy_actions_set_dict=collections.defaultdict()
    
    """loop through all the data sheets"""
    for sheet_name, sheet in data_sheet_dict.items():
        #read in the cells containing energy data
        #select the column first: 
        data_energy_col=sheet.col_values(2,1) #"1" here skips the 0th element (the header in this case)
        #select the label column
        data_energy_label=sheet.col_values(4,1) 
        #reset something
        data_parsed_temp=collections.defaultdict(list)
        data_energy_actions_set=set()
        #attach label to the data column: whether a data entry is "process only" or "embedded"
        for i in range (len(data_energy_col)):
            if data_energy_label[i]==0:
                data_energy_col[i]+=','+"Proc"
            else:
                data_energy_col[i]+=','+"Embed"
        #split the string of each cell
        for cell in data_energy_col:
            temp=[str(i) for i in cell.split(",")] #change each split element from unicode to str
            temp[2]=float(temp[2]) #change the actual data to float type
            data_energy_actions_set.add(temp[1])
        #reformat and append the information to each key
            data_parsed_temp[str(temp[0]).upper()+","+str(temp[1]).upper()+","+str(temp[-1]).upper()].append(temp[2])
        #get min and max value for each key in the dict
        for key, value in data_parsed_temp.items():
            max_=reduce(lambda a,b: a if a>b else b,value) #you need 'reduce' to do the iteration, althought it does not go into nested lists
            min_=reduce(lambda a,b: a if a<b else b,value)
            data_parsed_temp[key]=[min_,max_]
        #assign parsed data to corresponding dicts
        data_parsed_dict[sheet_name]=data_parsed_temp
        data_energy_actions_set_dict[sheet_name]=data_energy_actions_set
    return data_parsed_dict,data_energy_actions_set_dict



### unique_data_expansion function

In [ ]:
def unique_data_expansion(EO_basic,EO_extended_blank,mapping_dict):
    """This function copys the process energy data of unique category (e.g., 'Al hot rolled wrought') and paste it to all the relevant subcategories (e.g., 'Al 1070A hot rolled wrought')
    ## Input variables
    -EO_basic: a dataframe that contains the energy data for basic (unique) processes; dimension=(1,# of unique processes) or (# of combinations due to variation in energy data,# of unique processes)
    -EO_extended_blank: a dataframe that contains the zeros for all processes in y or PIOT; dimension=(1, total # of processes) or (# of combinations due to variation in energy data,total # of processes)
    -mapping_dict: contains {index_basic_EO: list[index1_extend_EO, index2_extend_EO...)}
    ## Output variables
    -EO_extended: a dataframe that contains the energy data for all processes in y or PIOT; dimension=(1, total # of processes) or (# of combinations due to variation in energy data,total # of processes)
    """
    
    ##loop through the mapping dict
    for col_basic, list_col_index_ext in mapping_dict.items():
        for col_index_ext in list_col_index_ext:
            EO_extended_blank.iloc[:,col_index_ext]=EO_basic.iloc[:,col_basic].values[0] #add '.value[0]' because now you only have one row of EO
    ##make a copy to output variable
    EO_extended=EO_extended_blank
    
    return EO_extended

### Data parsing module

In [ ]:
def Data_parsing_MOD(metal_process_data_workbook,body_composition_workbook,num_new_vehicles_dict,num_retired_vehicles_dict,time_span_tuple,compnt_loc_tuple_list,max_or_min):
    """This module does data parsing for the following output variables:
    ##Input variables
    -metal_process_data_workbook: contains worksheets of "Primary", "Manu" and "EoL treatment" that contain process data
    -body_composition_workbook: contains worksheet of the vehicle body composition data
    -num_new_vehicles_dict: a dict contains the new vehicle demand, in the format of {"vehicle type":[{year1:amount1},{year2:amount2}..]}
    -num_retired_vehicles_dict: a dict contains the # of vehicle retired, in the format of {"vehicle type":[{year1:amount1},{year2:amount2}..]}
    -time_span_tuple: a tuple contains the starting and ending years of the time span for modeling, identical to what's specified in MFA module    
    -compnt_loc_tuple_list: store the start and end rows (both inclusive) of a compnt in material composition tab "Alloy_compnt_G&L"
    -max_or_min: a boolean variable to decide whether use the max or min values of EO for subsequent calculations
    ## Output variables
    -PIOT_A_sheet: a dataframe of intersectoral coefficients, nxn matrix
    -ES_sheet: a dataframe of energy source share (in fraction) for each sector, mxn matrix
    -eff_sheet: energy use efficiency for each energy source, mx1 vector
    -EF_sheet: emission factors for each energy source, 1xm row vector
    -energy_data_parsed_overall_dict: a dictonary of parsed process energy data for stages of interest,in the format of {"stage name": {"metal name processed,process,unit":[min,max]}}
    -energy_activity_parsed_overall_dict: a dictonary of parsed process name for stages of interest,in the format of {"stage name": (process)}
    -EO_sheet_final_extended: a dataframe with col_name being materials sectors and row being process energy data (max or min) for each sector
                                in the shape of (1, n)   
    -data_body_composition_parsed: material composition of vehicles by type, in the format of {'vehi_type':[{material:mass}]}
    -y_total_alloy_time_series_tensor: a 4D tensor that contains total alloys demand (n) by component (j) for each vehicle type (channel 1=c), for each year (channel 2=z), kg
    -y_total_scrap_potential_time_series_tensor: a 4D tensor that contains total potential scrap alloys available (n) by component (j) for each vehicle type (channel 1=c), for each year (channel 2=t), kg
    -alloy_AE_spec_matrix: a matrix of alloys (row) and their AE specs (col), in %, (n,z)
    -mat_compnt_tensor: materials (row) by components (col) matrix that shows the material composition of each component, 
       in normalized mass (kg/kg), (n,j), for all vehicle types (c), (n,j,c)
    -ingot_alloy_loss_matrix: a matrix of alloys from secondary (row) and corresponding cumulative loss from 'ingot' stage to 'alloy' stage, (n,1)
    -TD_matrix: thermodynmic limit matrix that contains the yield of AE (col) during remelting of scrap allys of same group (row),
        in %, (y,z)
    -total_new_vehi_time_series_tensor: total number of vehicles (row=1) by vehicle type (col=c) by year, (1,c,t)
    
    ###Caution:
    *There is a "compnt_loc_tuple_list" variable that is hard-coded to store the start and end rows (both inclusive) of a compnt
    *check for "====hard coding warning===="
    *mixed use of 'z' and 't' for time dimension 
    """
    
    
    """Load data files"""
    ##load in workbooks named "data_input_lightweight","data_input_battery","data_input_composition"
    data_metal_process_loaded=xlrd.open_workbook(metal_process_data_workbook)
    body_composition_loaded=xlrd.open_workbook(body_composition_workbook)
    #for material manufacturing and EoL
    primary_part_sheet=data_metal_process_loaded.sheet_by_name("Primary_part") #get to the actual tab
    manu_part_sheet=data_metal_process_loaded.sheet_by_name("Manu_part")
    EoL_part_sheet=data_metal_process_loaded.sheet_by_name("EoL_part")
    #for composition
    body_composition_sheet=body_composition_loaded.sheet_by_name("Alloy_compnt_G&L")
    body_composition_percent_sheet=body_composition_loaded.sheet_by_name("Alloy_compnt_G&L_percent")
    #body_composition_sheet_virgin=body_composition_loaded.sheet_by_name("Archetype_body_GREET2_(virgin)") #no scrap reuse
    #for AE spec of alloys
    AE_spec_sheet=body_composition_loaded.sheet_by_name("Alloy_spec")

    """Parse PIOT_A, ES, eff, EF"""
    #use pandas here, in the future, may need to reconcile with xlrd
    xls_file=pd.ExcelFile(metal_process_data_workbook)
    #get PIOT_A
    PIOT_A_sheet=pd.read_excel(xls_file,"PIOT_A",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    print ("dimension of PIOT_A is: ",PIOT_A_sheet.shape,"\n")
    #get ES
    ES_sheet=pd.read_excel(xls_file,"PIOT_ES",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    print ("dimension of ES is: ",ES_sheet.shape,"\n")
    #get eff
    eff_sheet=pd.read_excel(xls_file,"PIOT_eff",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    eff_sheet.drop(["ref","ref2","Energy use efficiency","Life cycle energy"],inplace=True,axis=1)
    print ("dimension of eff is: ",eff_sheet.shape,"\n")
    #get EF
    EF_sheet=pd.read_excel(xls_file,"PIOT_EF",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    EF_sheet.drop("ref",inplace=True,axis=1) #if inplace=False (default), the original EF_sheet will not be changed, only copy will be altered
    EF_sheet=EF_sheet.T
    print ("dimension of EF is: ",EF_sheet.shape,"\n")
    
    #for 'ingot' to 'alloy' loss factors
    ingot_alloy_loss_matrix=pd.read_excel(xls_file,"Ingot_alloy_loss_matrix",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    print ("dimension of ingot-alloy-loss matrix is: ",ingot_alloy_loss_matrix.shape,"\n")
    #print ("values of ingot-alloy-loss matrix: ", ingot_alloy_loss_matrix,"\n")

    #for 'TD_filter'
    TD_matrix=pd.read_excel(xls_file,"TD_filter",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    #print ("TD_matrix first import: (shpae)", TD_matrix.shape,"\n")
    #print ("TD_matrix first import: (headers)", TD_matrix.columns,"\n")
           
    """Parse vehicle composition data"""
    #a dictionary to store the reformated data
    data_body_composition_parsed=collections.defaultdict()
    #read in the cells containing body composition data
    #get a list of vehicle types, components
    temp_vehicle_types=[]
    temp_vehicle_compnts=set() 
    
    #====hard coding warning====#
    num_vehi_cols=body_composition_sheet.ncols-15 #15 is number of columns that are (necessary but) not vehicle
    
    #new for material composition by components: a list to store the start and end rows (both inclusive) of a compnt
    #now an input variable
    #compnt_loc_tuple_list=[(2,61), (63,122),(124,183),(185,244),(246,305),(307,366),(368,427)]

    #prepare dimension for alloys used a vehicle, compnts and vehi types; for number of rows for alloys, need to do a calculation
    dim_nc,dim_j,dim_c=int((body_composition_sheet.nrows-1)/len(compnt_loc_tuple_list)-1),int(len(compnt_loc_tuple_list)),int(num_vehi_cols)
    """caution!!! total metals (rows=ny) for y tensor (defined later in the module) is larger than total alloys (rows=nc) for vehicle components!!!!"""
    
    #create a vehicle composition tensor (nc,j,c)
    alloy_compnt_vehi_tensor=np.zeros((dim_nc,dim_j,dim_c))
    #print alloy_compnt_vehi_tensor.shape

    ##create a dict of entire composition table of all vehi types  
    #format:{'vehi_type':{compnt:[{'steel hot rolled stamped': xxx kg}, {'steel cold rolled stamped': yyy kg}...] } }
    for i in range(num_vehi_cols): 
        temp_vehicle_types.append(str(body_composition_sheet.cell_value(0,i+1))) 
        #for each index (vehi type), further create a dict of lists to store material data
        data_body_composition_parsed[temp_vehicle_types[i]]=collections.defaultdict(list)
        for compnt_index, compnt_tuple in enumerate (compnt_loc_tuple_list):
            #create a index entry with corresponding compnt name, and keep assigning values of materials to its list 
            temp_vehicle_compnts.add(str(body_composition_sheet.cell_value(compnt_tuple[0]-1,0)))
            for alloy_index, j in enumerate (range(compnt_tuple[0],compnt_tuple[1]+1)):
                #data_body_composition_parsed[temp_vehicle_types[i-1]].append({str(body_composition_sheet.cell_value(j,0)):float(body_composition_sheet.cell_value(j,i))})    
                data_body_composition_parsed[temp_vehicle_types[i]][str(body_composition_sheet.cell_value(compnt_tuple[0]-1,0))].append({str(body_composition_sheet.cell_value(j,0)):float(body_composition_sheet.cell_value(j,i+1))})
                #popluate the "alloy composition by compnt by vehi type" tensor
                alloy_compnt_vehi_tensor[alloy_index,compnt_index,i]=float(body_composition_sheet.cell_value(j,i+1))
                    
    #create the alloy_AE_sepc_matrix (#alloys,#AE)
    dim_AE=AE_spec_sheet.ncols-1
    alloy_AE_spec_matrix=np.zeros((dim_nc,dim_AE))
    for i in range (dim_nc):
        for j in range (dim_AE):
            alloy_AE_spec_matrix[i,j]=float(AE_spec_sheet.cell_value(i+1,j+1)) #0-index, row=0 is the header

    #create mat_compnt_tensor (kg alloy/kg component,#component,#vehi_type)
    mat_compnt_tensor=np.zeros((dim_nc,dim_j,dim_c))
    for l in range (dim_c):
        for j in range (dim_j):
            for i in range (dim_nc):
                #first two rows are: overal header (vehicle types) and subheader of 1st compont
                mat_compnt_tensor[i,j,l]=float(body_composition_percent_sheet.cell_value(i+2+j*(dim_nc+1),l+1))
    
    #print ("percentage of Steel galvanized stamped in power system in lightweight ICEV: ",mat_compnt_tensor[2,1,1],"\n")
    
    """Parse process energy data"""
    #prepare the data sheet dict for the function
    data_sheet_input_dict={'Primary_part':primary_part_sheet,'Manu_part':manu_part_sheet,'EoL_part':EoL_part_sheet}
    energy_data_parsed_overall_dict,energy_activity_parsed_overall_dict=metal_data_parsing(data_sheet_input_dict)

       
    """create dataframes to for creating EO"""
    ##create dataframe for each stage from dict, using min values
    temp_dict_min=collections.defaultdict(float)
    for sheet_name in data_sheet_input_dict.keys():
        for key,value in energy_data_parsed_overall_dict[sheet_name].items():
            if value[0] < temp_dict_min[key.split(",")[0]] or not temp_dict_min[key.split(",")[0]]:
                temp_dict_min[key.split(",")[0]]=value[0] 

    #CAUTION: the order of the data entries are based on hash value, NOT the order in your PIOT.
    df_process_energy_min_concat=pd.DataFrame.from_dict(temp_dict_min,orient="index") 
    df_process_energy_min_concat.columns=['Process energy (MJ/kg)']          
                
    ##create dataframe for each stage from dict, using max values
    temp_dict_max=collections.defaultdict(float)
    for sheet_name in data_sheet_input_dict.keys():
        for key,value in energy_data_parsed_overall_dict[sheet_name].items():
            if value[1] > temp_dict_max[key.split(",")[0]] or not temp_dict_max[key.split(",")[0]]:
                temp_dict_max[key.split(",")[0]]=value[1] 

    #CAUTION: the order of the data entries are based on hash value, NOT the order in your PIOT.
    df_process_energy_max_concat=pd.DataFrame.from_dict(temp_dict_max,orient="index") 
    df_process_energy_max_concat.columns=['Process energy (MJ/kg)']
   

    """Parse demand data (for y tensor) from MFA module"""
    #get a copy of the sheet with all materials (not just alloys in vehicles, but also intermediates such as "steel hot rolled")
    y_sheet=pd.read_excel(xls_file,"PIOT_y_by_compnt",usecols=lambda x: 'Unnamed' not in x) #remove the first col
    #reset all values to zero
    y_sheet[:]=0.0  
    #create a dim for rows of y sheet: number of rows are more than what's in vehicle composition sheet, 
    #  as you have intermediate materials (e.g. aluminum ingot) that are not present in vehicle composition
    #also create a dim for time span
    dim_ny, dim_z=y_sheet.shape[0], time_span_tuple[1]-time_span_tuple[0]+1
    #record the total time span (current time span could be a fraction of"total time span" from the MFA)
    #e.g., total tim span can be 1999 to 2050, while current time span could be 2017 to 2030
    #so, just pick one vehi type and get start and end years of total time span
    total_time_span_tuple=(list(num_new_vehicles_dict['PHEV_conventional'][0].keys())[0],list(num_new_vehicles_dict['PHEV_conventional'][-1].keys())[0])
    #then create a locator to locate the index corresponds to the year in the list of {year:vehi_vol}s
    year_index_locator=time_span_tuple[0]-total_time_span_tuple[0]
    
    ##populate the y tensors (ny,j,c,z) for total alloy demand and potential scrap availability (other rows remain zero)
    #create an empty tensor
    y_total_alloy_time_series_tensor=np.zeros((dim_ny,dim_j,dim_c,dim_z))
    y_total_scrap_potential_time_series_tensor=np.zeros((dim_ny,dim_j,dim_c,dim_z))
    total_new_vehi_time_series_tensor=np.zeros((1,dim_c,dim_z)) #total volume of new vehicle demand by vehicle type by year (sorry for the confusion, should use t for time)
       
    for vehi_index, vehi_type in enumerate (temp_vehicle_types):
        for year_index, year in enumerate (range(time_span_tuple[0],time_span_tuple[1]+1)):
            for compnt_index, compnt_type in enumerate (temp_vehicle_compnts):
                for alloy_index in range(alloy_compnt_vehi_tensor.shape[0]): #the first nc rows that corrspond to virgin alloys
                    #calculate material (total alloy only) demand from "num_new_vehicles_dict" 
                    y_total_alloy_time_series_tensor[alloy_index,compnt_index,vehi_index,year_index]=alloy_compnt_vehi_tensor[alloy_index,compnt_index,vehi_index]*num_new_vehicles_dict[vehi_type][year_index+year_index_locator][year]
                    #calculate the potential available scrap from "num_retired_vehicles_dict"; first row of scrap alloys starting after the last row of virgin alloys (i.e., nc+1)
                    y_total_scrap_potential_time_series_tensor[alloy_index+dim_nc,compnt_index,vehi_index,year_index]=alloy_compnt_vehi_tensor[alloy_index,compnt_index,vehi_index]*num_retired_vehicles_dict[vehi_type][year_index+year_index_locator][year]
            #re-organize annual new vehicle demand in to (1,c,z=t)
            total_new_vehi_time_series_tensor[:,vehi_index,year_index]=num_new_vehicles_dict[vehi_type][year_index+year_index_locator][year]
    #One more step for scrap: merge the potential Mg scraps to one entry
    y_total_scrap_potential_time_series_tensor[dim_nc*2-2,:,:,:]+=y_total_scrap_potential_time_series_tensor[dim_nc*2-1,:,:,:]
    y_total_scrap_potential_time_series_tensor[dim_nc*2-1,:,:,:]=0
        
    
    """generate two versions of EO here (all min, all max)"""
    ##Prepare variables for EO
    EO_sheet=pd.read_excel(xls_file,"PIOT_EO_basic",usecols=lambda x: 'Unnamed' not in x) #remove the first col, start with basic version
    EO_sheet[:]=0.0 #change NaNs to zeros
    
    #get a list of EO_sheet column headers in string
    EO_sheet_col_name_list=[str(col_name).upper() for col_name in EO_sheet.columns]
    #transpose the dataframe of process energy from Data_parsing_MOD
    df_process_energy_min_concat_trans=df_process_energy_min_concat.T
    df_process_energy_max_concat_trans=df_process_energy_max_concat.T

    ##filter out the data entries from 'df_process_energy_trans' that are not used in EO
    #create a col name list for 'df_process_energy_trans'
    df_process_energy_col_name_list=[str(col_name).upper() for col_name in df_process_energy_max_concat_trans.columns]
    #create a list of col name that does not exist in EO sheet
    no_use_col_name_list=[col_name for col_name in df_process_energy_col_name_list if not (col_name in EO_sheet_col_name_list)]
    #create a list of col name that are in EO sheet but missing from 'df_process_energy'
    missing_col_name_list=[col_name for col_name in EO_sheet_col_name_list if not (col_name in df_process_energy_col_name_list)]
    #now, drop the columns in df_process energy_trans that are not used for EO
    df_process_energy_min_concat_trans.drop(no_use_col_name_list,axis=1, inplace=True)
    df_process_energy_max_concat_trans.drop(no_use_col_name_list,axis=1, inplace=True)
    
    ##export 'df_process_energy_min_concat_trans' and 'df_process_energy_max_concat_trans' to excel
    df_process_energy_max_concat_trans.to_excel("check_max_proc_energy_47_col.xlsx")
    df_process_energy_min_concat_trans.to_excel("check_min_proc_energy_47_col.xlsx")
    ##[caution] the order of columns is different between 'df_process_energy_xxx_concat_trans' and 'EO_sheet_min/max' 
       
    EO_sheet_min=EO_sheet.copy()
    EO_sheet_min.rename(columns=str.upper,inplace=True) #need to change column name to upper case
    EO_sheet_max=EO_sheet.copy()
    EO_sheet_max.rename(columns=str.upper,inplace=True) #need to change column name to upper case
    for col in EO_sheet_min.columns:
        EO_sheet_min.loc[:,col]=df_process_energy_min_concat_trans.loc[:,col][0]
        EO_sheet_max.loc[:,col]=df_process_energy_max_concat_trans.loc[:,col][0]
        
    print ("dimension of EO_basic is: ", EO_sheet.shape,"\n")
    ## export 'EO_sheet_min' and 'EO_sheet_max' to excel
    EO_sheet_min.to_excel("EO_sheet_min.xlsx")
    EO_sheet_max.to_excel("EO_sheet_max.xlsx")
    
    if max_or_min=='max':
        EO_sheet_final=EO_sheet_max
    else:
        EO_sheet_final=EO_sheet_min
        
    
    """expand from basic version of PIOT_EO to full length version of PIOT_EO"""
    ##The EO compiled above contains the entry like "Al hot rolled wrought", now we need to expand this to relevant alloys
    ##It's essentially a problem of multiplying the value of a col for a given times
    
    ##create input variables to the function
    EO_extended_blank=pd.read_excel(xls_file,"PIOT_EO",usecols=lambda x: 'Unnamed' not in x) #remove the first col, create a full version with one row of zeros
    EO_extended_blank=pd.DataFrame(data=np.zeros((EO_sheet_final.shape[0],EO_extended_blank.shape[1]))) #expand rows to total number of combos
    temp_mapping_sheet=pd.read_excel(xls_file,"EO_basic_to_EO_all",usecols=lambda x: 'Unnamed' not in x) #remove the first col, create a copy of mapping tab
    #create the mapping dict
    mapping_dict=collections.defaultdict(list)
    #zip the "EO_basic_col_index" and "EO_full_col_index" columns
    zipped_col_index=zip(temp_mapping_sheet.loc[:,"Col_index in EO_basic"],temp_mapping_sheet.loc[:,"col_index in EO_all entries"])
    for col_index_tuple in zipped_col_index:
        mapping_dict[col_index_tuple[0]].append(col_index_tuple[1])

    EO_sheet_final_extended=unique_data_expansion(EO_sheet_final,EO_extended_blank, mapping_dict)
    print ("dimension of EO (total ver): ", EO_sheet_final_extended.shape,"\n")
    ## export EO total ver for checking
    EO_sheet_final_extended.to_excel("EO_extended_ver.xlsx")

   
    return total_new_vehi_time_series_tensor,TD_matrix,ingot_alloy_loss_matrix,dim_ny, mat_compnt_tensor, alloy_AE_spec_matrix, data_body_composition_parsed,EO_sheet_final_extended,y_total_alloy_time_series_tensor,y_total_scrap_potential_time_series_tensor,PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet






### EoL module that explicitly deal with dismantling, shredding, sorting and blending

In [ ]:
def EoL_MOD(v_c_tensor,v_s_tensor,mat_compnt_tensor,AE_matrix,shredding_loss_rate,TD_matrix,alloy_group_cutoff,
            TargetAlloy_mass_by_compnt_by_vehi,ingot_alloy_loss_matrix,
            alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,dilution_explicit,
            rand_sort_error=0.02,cross_mixing_flag=True):
    """This module calculates the scrap generation from vehicle dismantling, shredding, sorting and blending. 
    Also calculated is the reuse amount of scrap and corresponding need for virgin bulk metals, AEs and virgin alloys 
    (that are still needed to fulfill the demand). 
    THE CALCULATION IS FOR ONE YEAR
    
    ## Input variables:
    ***Input variables below are for Dismantling***
    -v_c_tensor: a 3D tensor contains vectors containing the components (j) of a specific type of vehicle (1) for c types, in total mass (kg), (j,1,c)
    -v_s_tensor: a 3D tensor contains vectors containing the dismantling rate of each component (rest is left to "car hulk"), corresponding to v_c_tensor, in %, (j,1,c)
        note that "reuse/remanufacturing" (e.g., remanu of engine) is not considered; "dismantling rate" strictly refers to the portion that is "unwanted and shredded separately (from other components or hulk)"
    -mat_compnt_tensor: materials (row) by components (col) matrix that shows the material composition of each component, 
       in normalized mass (kg/kg), (n,j), for all vehicle types (c), (n,j,c)
    ***Input variables below are for Shredding***
    -AE_matrix: alloys (row) by alloying elements (col) matrix that shows the AE composition of each alloy, in %, (n,z)
    -shredding_loss_rate: a list contains the shredding loss rate for each component (len=j) or car hulk (len=1)
    ***Input variables below are for Sorting&Remelting***
    -TD_matrix: thermodynmic limit matrix that contains the yield of AE (col) during remelting of scrap allys of same group (row),
        in %, (y,z)
    -rand_sort_error: a random sorting error rate (scalar) that indicates the imperfect sorting 
    -alloy_group_cutoff: a list contains tuple of (staring,ending) rows in the alloy-related matrix that belongs to a certain common metal group,
        len()= #of metal types = y
    ***Input variables below are for Blending***
    -TargetAlloy_mass_by_compnt_by_vehi:a 4D tensor that contains target alloy by metal (row), by AE composition (col), 
        for each component OR car hulk (channel) and each vehicle type (channl2) in total mass (kg), (n,z,j,c)
    -ingot_alloy_loss_matrix: a matrix of alloys from secondary (row) and corresponding cumulative loss from 'ingot' stage to 'alloy' stage, (n,1)
    -dilution_explicit: a boolean variable to indicate whether "dilution (quality loss)" is explicitly modeled
    -closed_loop_recycl_rate_list: a list containing the metal group-lvl closed-loop recycling rate factors, in %
    -alpha: learning rate, scalar (dim=0)
    -num_iter: number of iterations before the gradient descent algorithm is terminated, scalar (dim=0)
    -epsilon: a small value to create margins of AE_spec%, in %
    -total_utility_score_goal: a small value to compare with total utility score of an iteration as the termination criteron , scalar (dim=0)


    ##Output variables: (OUTDATED)
    -virgin_bulk_metal_mass_by_compnt_output: a 3D tensor containing virgin bulk metals (row), by total mass (col), for each component OR car hulk (channel), in total mass (kg), (y,1,j)
    -virgin_AE_mass_by_compnt_output: a 3D tensor containing (1 row of) virgin AEs needed (col), for each component OR car hulk (channel), in total mass (kg), (1,z,j)
    -virgin_alloy_needed_by_compnt_output: a 3D tensor that stores the mass (col=1) of each target alloy from virgin sources(row),
        for each component OR car hulk (channel), in total mass (n,1,j)
    -Int_Target_mass_alloc_by_compnt_output: a 3D tensor that stores the mass of intermediate scrap alloy (col=1) allocated to each target alloy (row),
        for each component OR car hulk (channel), in total mass (n,1,j)
    -Int_Target_AE_alloc_by_compnt_output: a 3D tensor that stores the mass of AEs (including bulk) that constitute intermediate scrap alloy (col=z),
        allocated to each target alloy (row),for each component OR car hulk (channel), in total mass (n,z,j)
    -IntAlloy_leftover_by_compnt_output: a 3D tensor that stores the mass (col=1) of each intermediate scrap alloy that is not used up (row),
        for each component OR car hulk (channel), in total mass (y,1,j)
  
    ###KEYWORDS for NAMING CONVENTION
    "hulk": label variables where handling scraps as a whole hulk
    "SepCom": lable variables where handling scraps separately by individual component 
    "cross": lable variables where "cross-mixing" is assumed
    "no_cross": lable variables where "no cross-mixing" is assumed
    """
    
    
    
    """Common steps (Dismantling)"""
    ##Dismantling: this generates 3D tensors that contain mass of scraps by component for each (single) vehicle type
    #e.g., xx kg scrap "AL 1070A hot rolled wrought" from "transmission" of "BEV"
    scrap_compnt_to_SepCom_tensor,scrap_compnt_to_hulk_tensor=Dismantling(v_c_tensor,v_s_tensor,mat_compnt_tensor)
        
    
    """Arrange the following steps based on 'cross-mixing' or 'no cross-mixing' """
    if cross_mixing_flag:
        ##====Shredding====##
        #for dismantled compnts (the tensors are 3D (alloy,AE,compnt),the 4th dimension "vehi_type" has been collpased)
        shredded_alloys_mix_by_compnt_SepCom_cross,shred_loss_to_others_SepCom_cross,to_be_shredded_copy_SepCom_cross=Shredding(scrap_compnt_to_SepCom_tensor,AE_matrix,shredding_loss_rate,cross_mixing=cross_mixing_flag)
        #for hulk
        shredded_alloys_mix_by_compnt_hulk_cross,shred_loss_to_others_hulk_cross,to_be_shredded_copy_hulk_cross=Shredding(scrap_compnt_to_hulk_tensor,AE_matrix,shredding_loss_rate,cross_mixing=cross_mixing_flag)

        ##====Sorting&remelting====##
        """Only did for shreds from separated compnts, assume shred from hulk won't be used for closed-loop recycling"""
        IntAlloy_by_compnt_cross,remelt_loss_to_others_cross,to_be_remelted_copy_cross=Sorting_and_remelting(shredded_alloys_mix_by_compnt_SepCom_cross,alloy_group_cutoff,
                                                                                                             TD_matrix,rand_sort_error=rand_sort_error)

        
        """create the 'contribute_ratio' tensor to create the 4D tensor version of IntAlloy_by_compnt_cross"""
        #the IntAlloy_by_compnt_cross from Sorting&remelting step is 3D (y,z,j), because the dimension of vehicle types has been collapsed
        
        #initiate "contribute_ratio_SepCom_cross"
        contribute_ratio_SepCom_cross=np.ones((IntAlloy_by_compnt_cross.shape[0],IntAlloy_by_compnt_cross.shape[2],scrap_compnt_to_SepCom_tensor.shape[2])) #(y,j,c)
        for index,cutoff_tuple in enumerate(alloy_group_cutoff):
            temp_sum_shredded=np.sum(to_be_shredded_copy_SepCom_cross[cutoff_tuple[0]:cutoff_tuple[1]+1,:,:],axis=0,keepdims=True) #collapse k rows to one row, while keep the other two dimeions unchanged; (1,j,c)
            contribute_ratio_SepCom_cross[index,:,:]=np.divide(temp_sum_shredded,np.sum(temp_sum_shredded,axis=2,keepdims=True))
        #change 'nan' to zero; otherwise when you have zero mass of a metal group (e.g., Mg), you will have a 'nan' contribute ratio
        #which, after being multiplied with element in 'IntAlloy_by_compnt_cross tensor', will lead to 'nan' that cannot be plotted
        contribute_ratio_SepCom_cross=np.nan_to_num(contribute_ratio_SepCom_cross)
        
        #create the 4D tensor version of IntAlloy_by_compnt_cross (y,z,j,c)
        IntAlloy_by_compnt_cross=np.multiply(np.expand_dims(IntAlloy_by_compnt_cross,axis=3),np.expand_dims(contribute_ratio_SepCom_cross,axis=1))


        ##====Blending====##
        """similar to Sorting&remelting, only separated compnts are investigated here"""
        
        """create corresponding 4d tensors"""
        ##create 4D tensors to hold all the blending results of scraps of all vehicle types
        #tensors with alloys as rows, AE/total mass as col, compont as channel1, vehi_type as channel2
        blend_dim_n,blend_dim_z,blend_dim_j,blend_dim_c=(mat_compnt_tensor.shape[0],AE_matrix.shape[1],
                                                        mat_compnt_tensor.shape[1],mat_compnt_tensor.shape[2])
        Int_Target_mass_alloc_by_compnt_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        virgin_alloy_needed_by_compnt_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        virgin_alloy_offset_by_compnt_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        Int_Target_AE_alloc_by_compnt_cross=np.zeros((blend_dim_n,blend_dim_z,blend_dim_j,blend_dim_c))
        #tensors with metal group as rows, AE/total mass as col, compont as channel1, vehi_type as channel2
        blend_dim_y=TD_matrix.shape[0]
        #virgin_bulk_metal_mass_by_compnt_cross=np.zeros((blend_dim_y,1,blend_dim_j,blend_dim_c))
        IntAlloy_leftover_by_compnt_cross=np.zeros((blend_dim_y,blend_dim_z,blend_dim_j,blend_dim_c))
        #other tensors                            
        virgin_AE_mass_by_compnt_cross=np.zeros((1,blend_dim_z,blend_dim_j,blend_dim_c)) 
        IntAlloy_mass_alloc_by_compnt_cross=np.zeros((blend_dim_y,1,blend_dim_j,blend_dim_c))

                
        """generate outputs from Blending step"""
        for vehi_id in range (blend_dim_c):
            IntAlloy_mass_alloc_by_compnt_cross_temp,virgin_alloy_offset_by_compnt_cross_temp,virgin_alloy_needed_by_compnt_cross_temp,virgin_AE_mass_by_compnt_cross_temp,IntAlloy_leftover_by_compnt_cross_temp,Int_Target_mass_alloc_by_compnt_cross_temp,Int_Target_AE_alloc_by_compnt_cross_temp=Blending(IntAlloy_by_compnt_cross[:,:,:,vehi_id],TargetAlloy_mass_by_compnt_by_vehi[:,:,:,vehi_id],
                                                                AE_matrix,alloy_group_cutoff,ingot_alloy_loss_matrix,alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,IntAlloy_mass_alloc='Euclidean distance',dilution_explicit=dilution_explicit)

            #assign temp values to the intermdiate tensors
            IntAlloy_mass_alloc_by_compnt_cross[:,:,:,vehi_id]=IntAlloy_mass_alloc_by_compnt_cross_temp
            virgin_alloy_offset_by_compnt_cross[:,:,:,vehi_id]=virgin_alloy_offset_by_compnt_cross_temp
            virgin_alloy_needed_by_compnt_cross[:,:,:,vehi_id]=virgin_alloy_needed_by_compnt_cross_temp
            #virgin_bulk_metal_mass_by_compnt_cross[:,:,:,vehi_id]=virgin_bulk_metal_mass_by_compnt_cross_temp
            virgin_AE_mass_by_compnt_cross[:,:,:,vehi_id]=virgin_AE_mass_by_compnt_cross_temp
            IntAlloy_leftover_by_compnt_cross[:,:,:,vehi_id]=IntAlloy_leftover_by_compnt_cross_temp #this should show a decreasing trend along vehicle type id
            Int_Target_AE_alloc_by_compnt_cross[:,:,:,vehi_id]=Int_Target_AE_alloc_by_compnt_cross_temp
            Int_Target_mass_alloc_by_compnt_cross[:,:,:,vehi_id]=Int_Target_mass_alloc_by_compnt_cross_temp            
  

        ##Assign final outputs: 4D tensors
        IntAlloy_mass_alloc_by_compnt_output=IntAlloy_mass_alloc_by_compnt_cross
        virgin_alloy_offset_by_compnt_output=virgin_alloy_offset_by_compnt_cross
        virgin_alloy_needed_by_compnt_output=virgin_alloy_needed_by_compnt_cross
        #virgin_bulk_metal_mass_by_compnt_output=virgin_bulk_metal_mass_by_compnt_cross
        virgin_AE_mass_by_compnt_output=virgin_AE_mass_by_compnt_cross
        IntAlloy_leftover_by_compnt_output=IntAlloy_leftover_by_compnt_cross
        Int_Target_AE_alloc_by_compnt_output=Int_Target_AE_alloc_by_compnt_cross
        Int_Target_mass_alloc_by_compnt_output=Int_Target_mass_alloc_by_compnt_cross

 
    else:
        ##have to use a for loop to go through each vehicle type and sum up the variable of interest in the end
        
        ##====Shredding====##
        ##create a 4D tensor to hold all the shreded scraps of all vehicle types
        shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c=(AE_matrix.shape[0],AE_matrix.shape[1],
                                                        scrap_compnt_to_SepCom_tensor.shape[1],scrap_compnt_to_SepCom_tensor.shape[2])
        shredded_alloys_mix_by_compnt_SepCom_no_cross=np.zeros((shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c))
        shred_loss_to_others_SepCom_no_cross=np.zeros((shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c))
        shredded_alloys_mix_by_compnt_hulk_no_cross=np.zeros((shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c))
        shred_loss_to_others_hulk_no_cross=np.zeros((shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c))

        for vehi_type_id in range(scrap_compnt_to_SepCom_tensor.shape[2]):
            #for dismantled compnts
            shredded_alloys_mix_by_compnt_SepCom,shred_loss_to_others_SepCom,_=Shredding(scrap_compnt_to_SepCom_tensor[:,:,vehi_type_id],AE_matrix,shredding_loss_rate,cross_mixing=cross_mixing_flag)
            shredded_alloys_mix_by_compnt_SepCom_no_cross[:,:,:,vehi_type_id]=shredded_alloys_mix_by_compnt_SepCom[:,:,:]
            shred_loss_to_others_SepCom_no_cross[:,:,:,vehi_type_id]=shred_loss_to_others_SepCom[:,:,:]
            #for hulk
            shredded_alloys_mix_by_compnt_hulk,shred_loss_to_others_hulk,_=Shredding(scrap_compnt_to_hulk_tensor[:,:,vehi_type_id],AE_matrix,shredding_loss_rate,cross_mixing=cross_mixing_flag)
            shredded_alloys_mix_by_compnt_hulk_no_cross[:,:,:,vehi_type_id]=shredded_alloys_mix_by_compnt_hulk[:,:,:]
            shred_loss_to_others_hulk_no_cross[:,:,:,vehi_type_id]=shred_loss_to_others_hulk[:,:,:]            
          
        ##====Sorting&remelting====##
        ##create a 4D tensor to hold all the sorted&remelted scraps of all vehicle types
        sr_dim_y,sr_dim_z,sr_dim_j,sr_dim_c=(TD_matrix.shape[0],TD_matrix.shape[1],shred_dim_j,shred_dim_c)
        IntAlloy_by_compnt_no_cross=np.zeros((sr_dim_y,sr_dim_z,sr_dim_j,sr_dim_c))
        remelt_loss_to_others_no_cross=np.zeros((sr_dim_y,sr_dim_z,sr_dim_j,sr_dim_c))
        to_be_remelted_copy_no_cross=np.zeros((sr_dim_y,sr_dim_z,sr_dim_j,sr_dim_c))
        
        for vehi_type_id in range(sr_dim_c): #input to be sorted is a 4D tensor
            IntAlloy_by_compnt,remelt_loss_to_others,to_be_remelted_copy=Sorting_and_remelting(shredded_alloys_mix_by_compnt_SepCom_no_cross[:,:,:,vehi_type_id],alloy_group_cutoff,
                                                                                                             TD_matrix,rand_sort_error=rand_sort_error)
            IntAlloy_by_compnt_no_cross[:,:,:,vehi_type_id]=IntAlloy_by_compnt[:,:,:]
            remelt_loss_to_others_no_cross[:,:,:,vehi_type_id]=remelt_loss_to_others[:,:,:]
            to_be_remelted_copy_no_cross[:,:,:,vehi_type_id]=to_be_remelted_copy[:,:,:]
        
        ##====Blending====##
        """similar to Sorting&remelting, only separated compnts are investigated here"""
        ##create 4D tensors to hold all the blending results of scraps of all vehicle types
        #tensors with alloys as rows, AE/total mass as col, compont as channel1, vehi_type as channel2
        blend_dim_n,blend_dim_z,blend_dim_j,blend_dim_c=(shred_dim_n,shred_dim_z,shred_dim_j,shred_dim_c)
        Int_Target_mass_alloc_by_compnt_no_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        virgin_alloy_needed_by_compnt_no_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        virgin_alloy_offset_by_compnt_no_cross=np.zeros((blend_dim_n,1,blend_dim_j,blend_dim_c))
        Int_Target_AE_alloc_by_compnt_no_cross=np.zeros((blend_dim_n,blend_dim_z,blend_dim_j,blend_dim_c))
        #tensors with metal group as rows, AE/total mass as col, compont as channel1, vehi_type as channel2
        blend_dim_y,blend_dim_z,blend_dim_j,blend_dim_c=(sr_dim_y,sr_dim_z,sr_dim_j,sr_dim_c)
        #virgin_bulk_metal_mass_by_compnt_no_cross=np.zeros((blend_dim_y,1,blend_dim_j,blend_dim_c))
        IntAlloy_leftover_by_compnt_no_cross=np.zeros((blend_dim_y,blend_dim_z,blend_dim_j,blend_dim_c))
        #other tensors                            
        virgin_AE_mass_by_compnt_no_cross=np.zeros((1,blend_dim_z,blend_dim_j,blend_dim_c))   
        IntAlloy_mass_alloc_by_compnt_no_cross=np.zeros((blend_dim_y,1,blend_dim_j,blend_dim_c))
        
        for vehi_type_id in range(blend_dim_c):
            IntAlloy_mass_alloc_by_compnt,virgin_alloy_offset_by_compnt,virgin_alloy_needed_by_compnt,virgin_AE_mass_by_compnt,IntAlloy_leftover_by_compnt,Int_Target_mass_alloc_by_compnt,Int_Target_AE_alloc_by_compnt=Blending(IntAlloy_by_compnt_no_cross[:,:,:,vehi_type_id],TargetAlloy_mass_by_compnt_by_vehi[:,:,:,vehi_type_id],
                                                                                                                                                                                                 AE_matrix,alloy_group_cutoff,ingot_alloy_loss_matrix,alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,IntAlloy_mass_alloc='Euclidean distance',dilution_explicit=dilution_explicit)
            IntAlloy_mass_alloc_by_compnt_no_cross[:,:,:,vehi_type_id]=IntAlloy_mass_alloc_by_compnt[:,:,:]
            virgin_alloy_offset_by_compnt_no_cross[:,:,:,vehi_type_id]=virgin_alloy_offset_by_compnt[:,:,:]
            virgin_alloy_needed_by_compnt_no_cross[:,:,:,vehi_type_id]=virgin_alloy_needed_by_compnt[:,:,:]
            #virgin_bulk_metal_mass_by_compnt_no_cross[:,:,:,vehi_type_id]=virgin_bulk_metal_mass_by_compnt[:,:,:]
            virgin_AE_mass_by_compnt_no_cross[:,:,:,vehi_type_id]=virgin_AE_mass_by_compnt[:,:,:]
            IntAlloy_leftover_by_compnt_no_cross[:,:,:,vehi_type_id]=IntAlloy_leftover_by_compnt[:,:,:]
            Int_Target_AE_alloc_by_compnt_no_cross[:,:,:,vehi_type_id]=Int_Target_AE_alloc_by_compnt[:,:,:]
            Int_Target_mass_alloc_by_compnt_no_cross[:,:,:,vehi_type_id]=Int_Target_mass_alloc_by_compnt[:,:,:]
        
        
        ##Assign final outputs: 4D tensors
        IntAlloy_mass_alloc_by_compnt_output=IntAlloy_mass_alloc_by_compnt_no_cross       
        virgin_alloy_offset_by_compnt_output=virgin_alloy_offset_by_compnt_no_cross
        virgin_alloy_needed_by_compnt_output=virgin_alloy_needed_by_compnt_no_cross
        #virgin_bulk_metal_mass_by_compnt_output=virgin_bulk_metal_mass_by_compnt_no_cross
        virgin_AE_mass_by_compnt_output=virgin_AE_mass_by_compnt_no_cross
        IntAlloy_leftover_by_compnt_output=IntAlloy_leftover_by_compnt_no_cross
        Int_Target_AE_alloc_by_compnt_output=Int_Target_AE_alloc_by_compnt_no_cross
        Int_Target_mass_alloc_by_compnt_output=Int_Target_mass_alloc_by_compnt_no_cross
        

    
    return IntAlloy_mass_alloc_by_compnt_output,virgin_alloy_offset_by_compnt_output,virgin_alloy_needed_by_compnt_output,virgin_AE_mass_by_compnt_output,IntAlloy_leftover_by_compnt_output,Int_Target_mass_alloc_by_compnt_output,Int_Target_AE_alloc_by_compnt_output
    


### LCA_MOD for multiple components

In [ ]:
def LCA_MOD_multi(PIOT_A,ES,eff,EF,y_time_series_tensor,EO_sheet_final_extended,eff_consider=True):
    """This module calculate the inventory and emissions for products from primary and secondary manufacturing and EOL stages
    ## Input variables
    -PIOT_A: a matrix of intersectoral coefficients; (n,n)
    -ES: a matrix of energy source share (in fraction) for each sector; (m,n)
    -eff: energy use efficiency for each energy source; (m,1)
    -eff_consider: a boolean variable determining whether eff vector will be used in the calculation, set to False by default
    -EF: emission factors for each energy source; (1,m)
    -y_time_series_tensor: used for multiple purposes (below)
        a 4D tensor that contains materials (n) by component (j) for each vehicle type (channel 1=c), for each year (channel 2=t) 
        # for virgin alloys: a 4D tensor that contains virgin alloys demand (after offset) for each component (j) and vehicle type (c) for each year (t), in mass (kg), (ny,j,c,t)
            although the row# is n=ny, only the first nc rows (correspond to virgin alloys) may have postive values, the rest are zeros
        # for virgin bulk: a 4D tensor containing virgin bulk metals (row), for each component OR car hulk (j) for each vehicle type (c) for each year (t), in mass (kg), (ny,j,c,t)
            although the row# is n=ny, only the designated rows (corresponds to bulk metal primary production) may have positive values, the rest are zeros 
        # for virgin AE: a 4D tensor containing virgin AEs needed (row), for each component OR car hulk (j) fo each vehicle type (c) for each year (t), in mass (kg), (ny,j,c,t)
            although the row# is n=ny, only the designated rows (corresponds to AE primary production) may have positive values, the rest are zeros
        # for secondary alloys (blended, on-spec, to displace virgin ones): a 4D tensor that contains alloys (from secondary source) for each component (j) for each vehicle type (c) for each year (t), in mass (kg), (ny,j,c,t)
            although the row# is n=ny, only the second nc rows (correspond to alloys from secondary sources) may have positive values, the rest are zeros
        
    -EO_sheet_final_extended: a dataframe with col_name being materials sectors and row being process energy data (max or min) for each sector
                                in the shape of (1, n)   
    ## Output variables
    *may need to convert tensors into dicts
    -total_energy_all_years_dict: total energy consumption (in MJ), in the format of {'year1':[emission1,emission2...]...}
    -total_energy_by_source_all_years_dict: total energy consumption by source, in the format of {'year1':df1(total_energy_by source)...}
    -total_emissions_all_years_dict: total carbon emission, in the format of {'year1':[emission1,emission2...]...}
    -total_emissions_by_source_all_years_dict
    """
    
    
    """update variables for material and energy inventory calculation"""
    ##calculate total output for material demand for each year  
    #the x_tensor contains: material sector (n) by component (j) by vehicle type (c) by year (t)
    Leontief=(np.identity(len(PIOT_A.index))-PIOT_A)
    temp_inv=pd.DataFrame(data=inv(Leontief),index=PIOT_A.index,columns=PIOT_A.index) #(n,n)
    #note here we use 'm' instead of 'n' to respresent the 2nd axis, as you are not allowed to use the same letter to represent different axis for an tensor
    #even when both axes have the same dimension (both n material sectors), so accordingly the 1st axis of 2nd tensor is also representd by 'm'

    #print ("Mg demand (y) for all compont in light PHEV: ", np.sum(y_time_series_tensor[145,:,3,:]))
    
    total_output_x_tensor=np.einsum('nm,mjct->njct',temp_inv,y_time_series_tensor)
    #set all negative values to zero
    total_output_x_tensor[total_output_x_tensor<0]=0

    #show sum of individual AE
    print ("total Al for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[140,:,3,:]) )
    print ("total Si for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[141,:,3,:]) )
    print ("total Fe for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[142,:,3,:]) )
    print ("total Cu for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[143,:,3,:]) )
    print ("total Mn for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[144,:,3,:]) )
    print ("total Mg for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[145,:,3,:]) )
    print ("total Cr for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[147,:,3,:]) )
    print ("total Ni for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[148,:,3,:]) )
    print ("total Zn for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[149,:,3,:]) )
    print ("total Ti for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[150,:,3,:]) )
    print ("total Mo for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[151,:,3,:]) )
    print ("total C for ALL compnt in light PHEV: ",np.sum(total_output_x_tensor[152,:,3,:]) )
    
    ##calculate the normalized energy consumption per kg material, for all possible combinations of EO (m,n,x)
    energy_by_source_normal_tensor=np.einsum('mn,xn->mnx',ES,EO_sheet_final_extended)
    if eff_consider:
        energy_by_source_normal_tensor=np.einsum('mnx,ml->mnx', energy_by_source_normal_tensor,eff)

    ##scale up from normalized values to total values for each energy source (m), using the total output total_output_x_tensor, for vehicle type (c), all EO combos (x) and years (t); (m,c,x,t)
    total_energy_by_source_all_years_tensor=np.einsum('mnx,njct->mcxt',energy_by_source_normal_tensor,total_output_x_tensor)

    ##calculation emissions (m,c,x,t)
    total_emissions_by_source_all_years_tensor=np.einsum('lm,mcxt->mcxt', EF,total_energy_by_source_all_years_tensor)
    
    
    return total_emissions_by_source_all_years_tensor,total_energy_by_source_all_years_tensor

 

       

### Wrapper module

In [ ]:
def Wrapper_MOD(MFA_time_series_file,scenario_num,metal_process_data_workbook,body_composition_workbook,time_span_tuple,compnt_loc_tuple_list,
                v_s_tensor,shredding_loss_rate,alloy_group_cutoff,
                AE_row_list,Mg_Pigd_share_dict,remelt_scrap_row_list,
                alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,max_or_min,dilution_explicit,
                rand_sort_error=0.02,cross_mixing_flag=True,**kwargs):
    """This module configures the rest of the modules to perform the investigation of interest
    ## Input variables (excluding intermediate inputs from other modules)
    ***The input variables below are for MFA module***
    -MFA_time_series_file: a workbook contains data sheets of time series data
    -scenario_num: scenario number that reflects different assumptions regarding the MFA scenarios
    ***The input variables below are for Data parsing module***
    -metal_process_data_workbook: contains worksheets of "Primary", "Manu" and "EoL treatment" that contain process data
    -body_composition_workbook: contains worksheet of the vehicle body composition data
    -compnt_loc_tuple_list: store the start and end rows (both inclusive) of a compnt in material composition tab "Alloy_compnt_G&L"
    -time_span_tuple: a tuple contains the starting and ending years of the time span for modeling, identical to what's specified in MFA module    
    -max_or_min: a boolean variable to decide whether use the max or min values of EO for subsequent calculations
    ***The input variables below are for EoL module***
        **for Dismantling**
        -[intermediate variable now] v_c_tensor: a 3D tensor contains vectors containing the mass of components (j) of a specific type of vehicle (1) for c types, in total mass (kg), (j,1,c)
        -v_s_tensor: a 3D tensor contains vectors containing the dismantling rate of each component (rest is left to "car hulk"), corresponding to v_c_tensor, in %, (j,1,c)
            note that "reuse/remanufacturing" (e.g., remanu of engine) is not considered; "dismantling rate" strictly refers to the portion that is "unwanted and shredded separately (from other components or hulk)"
        -[intermediate variable now] mat_compnt_tensor: materials (row) by components (col) matrix that shows the material composition of each component, 
           in normalized mass (kg/kg), (n,j), for all vehicle types (c), (n,j,c)
        **for Shredding**
        -[intermediate variable now] AE_matrix: alloys (row) by alloying elements (col) matrix that shows the AE composition of each alloy, in %, (n,z)
        -shredding_loss_rate: a list contains the shredding loss rate for each component (len=j) or car hulk (len=1)
        **for Sorting&Remelting**
        -[intermediate variable now] TD_matrix: thermodynmic limit matrix that contains the yield of AE (col) during remelting of scrap allys of same group (row),
            in %, (y,z)
        -rand_sort_error: a random sorting error rate (scalar) that indicates the incidental 
        -alloy_group_cutoff: a list contains tuple of (staring,ending) rows in the alloy-related matrix that belongs to a certain common metal group,
            len()= #of metal types = y
        **for Blending**
        -[intermediate variable now] TargetAlloy_mass_by_compnt:a 3D tensor that contains target alloy by metal (row), by AE composition (col), 
            for each component OR car hulk (channel), in total mass (kg), (n,z,j)
        -dilution_explicit: a boolean variable to indicate whether "dilution (quality loss)" is explicitly modeled
        -closed_loop_recycl_rate_list: a list containing the metal group-lvl closed-loop recycling rate factors, in %
        -alpha: learning rate, scalar (dim=0)
        -num_iter: number of iterations before the gradient descent algorithm is terminated, scalar (dim=0)
        -epsilon: a small value to create margins of AE_spec%, in %
        -total_utility_score_goal: a small value to compare with total utility score of an iteration as the termination criteron , scalar (dim=0)

    ***The input variables below are for other sections of the code in Wrapper MOD***
    -AE_row_list: a list of row numbers that correspond to the location of AE entries
    -Mg_Pigd_share_dict: a dict contains row# of Mg from Pigdeon (key) in full length y vector and its market share (value), {row#: %Pigd}
    -remelt_scrap_row_list: a list of row numbers that correspond to the location of scrap remelted entries
    
    ## Output variables (excluding intermediate outputs from other modules)
    -
    
    ###KEYWORDS for NAMING CONVENTION
    #Dimenions
    'n': typically refer to the dimension of "alloys"
        sometimes also use "nc" (<n) to refer to alloy products that used in compnts (exluding intermediate alloys)
    'j': typically refere to the dimenion of "compnts"
    'z': typically refere to the dimenion of "AEs"
    'y': typically refere to the dimenion of "metal groups"
    'c': typically refere to the dimenion of "vehicle types"
    't': typically refere to the dimenion of "time (year)"

    """
    
    
    """Start with MFA to generate flow data of vehicles"""
    _,num_new_vehicles_dict,num_retired_vehicles_dict=MFA_MOD(MFA_time_series_file,scenario_num)
  
    
    """Run the flow data through Data parsing module"""
    total_new_vehi_time_series_tensor,TD_matrix,ingot_alloy_loss_matrix,dim_ny, mat_compnt_tensor, alloy_AE_spec_matrix, data_body_composition_parsed,EO_sheet_final_extended,y_total_alloy_time_series_tensor,y_total_scrap_potential_time_series_tensor,PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet=Data_parsing_MOD(metal_process_data_workbook,body_composition_workbook,num_new_vehicles_dict,num_retired_vehicles_dict,time_span_tuple,compnt_loc_tuple_list,max_or_min)
    
    
    ##create the v_c_year_tensor (4D): convert from y_total_scrap_potential_time_series_tensor (n,j,c,t)
    #CAUTION: the rows (n>nc) in y_total_scrap_potential_time_series_tensor contains NOT ONLY scrap alloys (where you have values)
    #         BUT ALSO the rows of intermediate products and virgin alloys (but you should have all zeros there)
    temp_convert_vc=np.sum(y_total_scrap_potential_time_series_tensor,axis=0,keepdims=True) #(1,j,c,t)
    v_c_year_tensor=np.einsum('ljct->jlct',temp_convert_vc)

    ##create the TargetAlloy_mass_by_compnt_year_tensor(4D): convert from y_total_alloy_time_series_tensor (n,j,c,t)
    #First, create a new dimension to expand mass of alloy to mass of AE: (n,j,c,t)—>(n,z,j,c,t)
    temp_convert_TA=np.expand_dims(y_total_alloy_time_series_tensor[:alloy_AE_spec_matrix.shape[0],:,:,:],axis=1) #(n,1,j,c,t)
    #then mulply mass of alloy with its corresponding spec
    temp_convert_TA=np.einsum('nljct,nz->nzjct', temp_convert_TA,alloy_AE_spec_matrix) #(n,z,j,c,t)

    TargetAlloy_mass_by_compnt_by_vehi_by_year_tensor=temp_convert_TA #this keeps the 'vehi_type' dimension (c) for the target alloys
      
    ##prepare other relevant input variables to EoL
    AE_matrix=alloy_AE_spec_matrix
    
    
    """Run the EoL module in a FOR loop to generate the virgin material demand (after offset) and treatment inventory for each year"""
    ##prepare the output variables for EoL module
    #material flows
    dim_nc,dim_j,dim_y,dim_t,dim_z=(alloy_AE_spec_matrix.shape[0],mat_compnt_tensor.shape[1],len(alloy_group_cutoff),y_total_alloy_time_series_tensor.shape[3],alloy_AE_spec_matrix.shape[1])
    
    #create a dimension for vehicle type
    dim_c=v_s_tensor.shape[2]
    """all the output variables (below) are updated to include a vehicle type dimension"""
    virgin_alloy_needed_by_compnt_year=np.zeros((dim_nc,1,dim_j,dim_c,dim_t))
    virgin_alloy_offset_by_compnt_year=np.zeros((dim_nc,1,dim_j,dim_c,dim_t))
    #virgin_bulk_metal_mass_by_compnt_year=np.zeros((dim_y,1,dim_j,dim_c,dim_t))
    IntAlloy_mass_alloc_by_compnt_year=np.zeros((dim_y,1,dim_j,dim_c,dim_t))
    virgin_AE_mass_by_compnt_year=np.zeros((1,dim_z,dim_j,dim_c,dim_t))
    IntAlloy_leftover_by_compnt_year=np.zeros((dim_y,dim_z,dim_j,dim_c,dim_t))
    Int_Target_AE_alloc_by_compnt_year=np.zeros((dim_nc,dim_z,dim_j,dim_c,dim_t))
    Int_Target_mass_alloc_by_compnt_year=np.zeros((dim_nc,1,dim_j,dim_c,dim_t))
 
    
    ##loop through the time span
    for year_index in range (time_span_tuple[1]-time_span_tuple[0]+1): #starts with year0, until yearX
        v_c_tensor=v_c_year_tensor[:,:,:,year_index]
        """need to carry the vehicle_type dimension"""
        TargetAlloy_mass_by_compnt_by_vehi=TargetAlloy_mass_by_compnt_by_vehi_by_year_tensor[:,:,:,:,year_index]
        
        IntAlloy_mass_alloc_by_compnt_output,virgin_alloy_offset_by_compnt_output,virgin_alloy_needed_by_compnt_output,virgin_AE_mass_by_compnt_output,IntAlloy_leftover_by_compnt_output,Int_Target_mass_alloc_by_compnt_output,Int_Target_AE_alloc_by_compnt_output=EoL_MOD(v_c_tensor,v_s_tensor,mat_compnt_tensor,AE_matrix,shredding_loss_rate,TD_matrix,
                                                                                                                                                                                                                                alloy_group_cutoff,TargetAlloy_mass_by_compnt_by_vehi,ingot_alloy_loss_matrix,alpha,num_iter,epsilon,total_utility_score_goal,closed_loop_recycl_rate_list,dilution_explicit=dilution_explicit,rand_sort_error=rand_sort_error,cross_mixing_flag=cross_mixing_flag)

        ##populate the outputs of the Wrapper
        #material flows
        IntAlloy_mass_alloc_by_compnt_year[:,:,:,:,year_index]=IntAlloy_mass_alloc_by_compnt_output[:,:,:,:]
        virgin_alloy_offset_by_compnt_year[:,:,:,:,year_index]=virgin_alloy_offset_by_compnt_output[:,:,:,:]
        virgin_alloy_needed_by_compnt_year[:,:,:,:,year_index]=virgin_alloy_needed_by_compnt_output[:,:,:,:]
        #virgin_bulk_metal_mass_by_compnt_year[:,:,:,:,year_index]=virgin_bulk_metal_mass_by_compnt_output[:,:,:,:]
        virgin_AE_mass_by_compnt_year[:,:,:,:,year_index]=virgin_AE_mass_by_compnt_output[:,:,:,:]
        IntAlloy_leftover_by_compnt_year[:,:,:,:,year_index]=IntAlloy_leftover_by_compnt_output[:,:,:,:]
        Int_Target_AE_alloc_by_compnt_year[:,:,:,:,year_index]=Int_Target_AE_alloc_by_compnt_output[:,:,:,:]


        
    """reformat the outputs to have ny rows so that they can be fed to LCA module"""
    ###need to "place“ the output tensors to the corresponding position of the tensor with ny rows###
    ##create a set of "zero tensor" with desired dimensions (e.g., change dim_nc to dim_ny that corresponds to the # of rows for y vectors)
    zeros_virgin_alloy_needed_by_compnt_year=np.zeros((dim_ny,1,dim_j,dim_c,dim_t)) #need to feed in numbers for the first dim_nc rows
    #zeros_virgin_bulk_metal_mass_by_compnt_year=np.zeros((dim_ny,1,dim_j,dim_c,dim_t)) #need to feed in numbers for rows of virgin bulk metals
    zeros_virgin_AE_mass_by_compnt_year=np.zeros((dim_ny,1,dim_j,dim_c,dim_t)) #need to feed in number for rows of AEs
    zeros_virgin_alloy_offset_by_compnt_year=np.zeros((dim_ny,1,dim_j,dim_c,dim_t)) #need to feed in number for the second dim_nc rows
    zeros_IntAlloy_mass_alloc_by_compnt_year=np.zeros((dim_ny,1,dim_j,dim_c,dim_t)) #need to feed in number for rows of scrap remelted
    
    ##assign values to the output variables to be used for LCA
    #for virgin alloy demand
    zeros_virgin_alloy_needed_by_compnt_year[0:dim_nc,:,:,:,:]=virgin_alloy_needed_by_compnt_year[:,:,:,:,:]
    to_LCA_virgin_alloy_needed_by_compnt_year=zeros_virgin_alloy_needed_by_compnt_year
        
    #similarly, loop through the row locator for AE entries
    #Also make sure the index matches between this list and what's in 'virgin_AE_mass_by_compnt_year' （make sure the order of AEs in this list is the same as the order in column names of AEs)
    #re-organize the tensor dimension to fit the dimension of output tensor
    temp_virgin_AE_mass_by_compnt_year=np.einsum('lzjct->zljct',virgin_AE_mass_by_compnt_year)
    
    for index,AE_row in enumerate(AE_row_list):
        zeros_virgin_AE_mass_by_compnt_year[AE_row,:,:,:,:]=temp_virgin_AE_mass_by_compnt_year[index,:,:,:,:]
    
    #similarly, deal with issue of having two Mg primary production technologies
    #could be combined with handling of Mg in virgin bulk metal, but leave as separated for clarity
    for key, value in Mg_Pigd_share_dict.items():
        #actually there should be only one pair of key and value...
        #the row of 'electrolytic' should always be above 'Pigdgeon'
        temp_Mg_alloc=deepcopy(zeros_virgin_AE_mass_by_compnt_year[key-1,:,:,:,:])*value    
        zeros_virgin_AE_mass_by_compnt_year[key,:,:,:,:]=temp_Mg_alloc
        #subtract the corresponding amount from row of 'electrolytic' 
        zeros_virgin_AE_mass_by_compnt_year[key-1,:,:,:,:]-=temp_Mg_alloc

    #assign the final result to output tensor (to LCA)
    to_LCA_virgin_AE_mass_by_compnt_year=zeros_virgin_AE_mass_by_compnt_year
    
    #for scrap alloy processing (there are energy associated with using scrap alloys to offset the virgin demand)
    zeros_virgin_alloy_offset_by_compnt_year[dim_nc:(dim_nc+dim_nc),:,:,:,:]=virgin_alloy_offset_by_compnt_year[:,:,:,:,:]
    to_LCA_virgin_alloy_offset_by_compnt_year=zeros_virgin_alloy_offset_by_compnt_year
    
    #for remelted scrap
    for index,remelt_row in enumerate(remelt_scrap_row_list):
        #make sure the sequence of 'remelt_row' matches the sequence of 'remelted scrap' in IntAlloy_mass_alloc_by_compnt_year
        zeros_IntAlloy_mass_alloc_by_compnt_year[remelt_row,:,:,:,:]=IntAlloy_mass_alloc_by_compnt_year[index,:,:,:,:]
    to_LCA_IntAlloy_mass_alloc_by_compnt_year=zeros_IntAlloy_mass_alloc_by_compnt_year
        
    ##reformat the output tensors (to LCA_MOD) ensure their dimensions all conform to (ny,j,c,t) for 'y_time_series tensor'
    to_LCA_virgin_alloy_needed_by_compnt_year=np.squeeze(to_LCA_virgin_alloy_needed_by_compnt_year)
    #to_LCA_virgin_bulk_metal_mass_by_compnt_year=np.squeeze(to_LCA_virgin_bulk_metal_mass_by_compnt_year)
    to_LCA_virgin_AE_mass_by_compnt_year=np.squeeze(to_LCA_virgin_AE_mass_by_compnt_year)
    to_LCA_virgin_alloy_offset_by_compnt_year=np.squeeze(to_LCA_virgin_alloy_offset_by_compnt_year)
    to_LCA_IntAlloy_mass_alloc_by_compnt_year=np.squeeze(to_LCA_IntAlloy_mass_alloc_by_compnt_year)
    
    """run LCA module"""
    ###run the LCA module multiple times for different y_time_series tensors
    ##from virgin alloy demand (only from ingot-to-alloy energy consumptions are captured)
    for_virgin_alloy_total_emissions_by_source_all_years_tensor,for_virgin_alloy_total_energy_by_source_all_years_tensor=LCA_MOD_multi(PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet,to_LCA_virgin_alloy_needed_by_compnt_year,EO_sheet_final_extended,eff_consider=True)
    
    ##from virgin AE demand for 1) blending with scrap ingot, 2) AE that consist of virgin alloys still needed
    for_virgin_AE_total_emissions_by_source_all_years_tensor,for_virgin_AE_total_energy_by_source_all_years_tensor=LCA_MOD_multi(PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet,to_LCA_virgin_AE_mass_by_compnt_year,EO_sheet_final_extended,eff_consider=True)
    
    ##from making alloy from secondary source
    #CAUTION: THIS ONLY COVERS energy consumption from "Blending" and beyond (inclusive of "Blending step" which is represente d by 'ingotting' step in PIOT)
    #i.e., input coefficients to metal ingots are zero in PIOT
    for_secondary_alloy_total_emissions_by_source_all_years_tensor,for_secondary_alloy_total_energy_by_source_all_years_tensor=LCA_MOD_multi(PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet,to_LCA_virgin_alloy_offset_by_compnt_year,EO_sheet_final_extended,eff_consider=True)
        
    ##from scrap remelting (automatically link 'scrap separation, shredding and dismentling' using PIOT)
    #use 'to_LCA_IntAlloy_mass_alloc_by_compnt_year' as demand for scrap remelted
    for_scrap_proc_before_ingot_total_emissions_by_source_all_years_tensor,for_scrap_proc_before_ingot_total_energy_by_source_all_years_tensor=LCA_MOD_multi(PIOT_A_sheet,ES_sheet,eff_sheet,EF_sheet,to_LCA_IntAlloy_mass_alloc_by_compnt_year,EO_sheet_final_extended,eff_consider=True)
    
    ###get an intermdiate sum of total energy and emissions
    ##keep original dimension (m,c,x,t), where m is the energy sources, x is the total # of combinations [min,max] values for EO
    intermediate_total_LC_energy=for_virgin_alloy_total_energy_by_source_all_years_tensor+for_virgin_AE_total_energy_by_source_all_years_tensor+ \
    for_secondary_alloy_total_energy_by_source_all_years_tensor+for_scrap_proc_before_ingot_total_energy_by_source_all_years_tensor
    intermediate_total_LC_emissions=for_virgin_alloy_total_emissions_by_source_all_years_tensor+for_virgin_AE_total_emissions_by_source_all_years_tensor+ \
    for_secondary_alloy_total_emissions_by_source_all_years_tensor+for_scrap_proc_before_ingot_total_emissions_by_source_all_years_tensor

    ##reduce the dimension to (c,x,t), then re-order to (x,c,t) so that later you can add additional impacts that has a dimension of (1,c,t)
    intermediate_total_LC_energy=np.sum(intermediate_total_LC_energy, axis=0)
    intermediate_total_LC_emissions=np.sum(intermediate_total_LC_emissions,axis=0)
    intermediate_total_LC_energy=np.einsum('cxt->xct',intermediate_total_LC_energy)
    intermediate_total_LC_emissions=np.einsum('cxt->xct',intermediate_total_LC_emissions)
    
    
    """account for impacts from other steps (kwargs of dict)"""
    ###the input should be multipliers that scale based on the vehicle production volume
    if kwargs is not None:
        for kw,value in kwargs.items():
            if kw=='assembly_energy_consumption_MJ_vehicle':
                #this kw should contain the assembly energy in MJ/vehicle
                #create empty tensor for total energy from vehicle assembly (1,c,t)
                total_energy_vehi_assembly_year=np.zeros((1,dim_c,dim_t))
                #calculate total energy from vehicle assembly
                total_energy_vehi_assembly_year=np.multiply(total_new_vehi_time_series_tensor,value)
                
                ##update total LC energy
                intermediate_total_LC_energy+=total_energy_vehi_assembly_year
            elif kw=='assembly_emissions_kgCO2eq_vehicle':
                #this kw should contain the assembly emission in MJ/vehicle
                #create empty tensor for total emissions from vehicle assembly (1,c,t)
                total_emissions_vehi_assembly_year=np.zeros((1,dim_c,dim_t)) 
                #calculate total emissions from vehicle assembly
                total_emissions_vehi_assembly_year=np.multiply(total_new_vehi_time_series_tensor,value)
                 
                ##update total emissions
                intermediate_total_LC_emissions+=total_emissions_vehi_assembly_year


    """Generate final results"""
    ###====material flows by year====### 
    ##virgin alloy (after offset) demand, AE, bulk (ny,c,t)
    final_virgin_alloy_demand=np.sum(to_LCA_virgin_alloy_needed_by_compnt_year,axis=1) #from (ny,j,c,t) to (ny,c,t)
    final_AE_demand=np.sum(to_LCA_virgin_AE_mass_by_compnt_year,axis=1) #from (ny,j,c,t) to (ny,c,t)
    #final_bulk_demand=np.sum(to_LCA_virgin_bulk_metal_mass_by_compnt_year,axis=1) #from (ny,j,c,t) to (ny,c,t)
    
    ##leftover (remelted) intermediate scrap alloy (y,c,t)
    final_leftover_IntAlloy=np.sum(IntAlloy_leftover_by_compnt_year,axis=1) #from (y,z,j,c,t) to (y,j,c,t)
    final_leftover_IntAlloy=np.sum(final_leftover_IntAlloy,axis=1) #from (y,j,c,t) to (y,c,t)
    
    ##allocated (utilized) intermediate scrap alloy (y,c,t)
    final_utilized_IntAlloy=np.squeeze(IntAlloy_mass_alloc_by_compnt_year) #from (y,1,j,c,t) to (y,j,c,t)
    final_utilized_IntAlloy=np.sum(final_utilized_IntAlloy,axis=1) #from (y,j,c,t) to (y,c,t)
    final_utilized_ratio=np.round(final_utilized_IntAlloy/(final_utilized_IntAlloy+final_leftover_IntAlloy)*100,2) #(y,c,t)
    
    ##offset (derived from IntAlloy) (nc,c,t)
    final_virgin_alloy_offset=np.squeeze(virgin_alloy_offset_by_compnt_year) #from (nc,1,j,c,t) to (nc,j,c,t)
    final_virgin_alloy_offset=np.sum(final_virgin_alloy_offset,axis=1) #from (nc,j,c,t) to (nc,c,t)
    
    ##virgin alloy demand after offset (nc,c,t)
    final_virgin_alloy_needed=np.squeeze(virgin_alloy_needed_by_compnt_year) #from (nc,1,j,c,t) to (nc,j,c,t)
    final_virgin_alloy_needed=np.sum(final_virgin_alloy_needed,axis=1) #from (nc,j,c,t) to (nc,c,t)
    
    ###====energy consumption by year (x,c,t)====###
    ##total life cycle energy consumption by vehicle type
    final_total_LC_energy=intermediate_total_LC_energy
    #print "final_total_LC_energy: ",final_total_LC_energy,"\n"
    
    ##normalized life cycle energy consumption by vehicle type (x,c,t)
    final_norm_LC_energy=np.divide(final_total_LC_energy,total_new_vehi_time_series_tensor) #dim=(x,c,t)/(1,c,t)
    
    ###====emissions by year (x,c,t)====###
    ##total life cycle emissions by vehicle type
    final_total_LC_emissions=intermediate_total_LC_emissions
    
    ##normalized life cycle emissions by vehicle type
    final_norm_LC_emissions=np.divide(final_total_LC_emissions,total_new_vehi_time_series_tensor) #dim=(x,c,t)/(1,c,t)
    
    ###secondary material reuse rate (y,c,t): e.g., %of Al alloy demand is by using scrap alloys (mass ratio)
    
    
    """test output zone"""
    #print "normalized LC energy (MJ/vehicle): ",final_norm_LC_energy,"\n"
    #print "mean of normalized LC energy (MJ/vehicle): ",np.mean(final_norm_LC_energy,axis=0),"\n"
    #print "dim of normalized LC energy (MJ/vehicle): ",final_norm_LC_energy.shape,"\n"
    
    ###material flows
    #print "utilized IntAlloy (all vehicles):",np.sum(final_utilized_IntAlloy,axis=1,keepdims=True),"\n" #(y,1,t),collapse overall all vehicle types and components
    #print "utilized IntAlloy (by vehicle type):",final_utilized_IntAlloy,"\n" #(y,c,t)  
    #print "dim of utilized IntAlloy: ", np.sum(final_utilized_IntAlloy,axis=1,keepdims=True).shape,"\n"
    #print "leftover IntAlloy by mass (all vehicles): ", np.sum(final_leftover_IntAlloy,axis=1,keepdims=True),"\n" #(y,1,t),collapse overall all vehicle types and components
    #print "leftover IntAlloy by mass (by vehicle type): ", final_leftover_IntAlloy,"\n"     
    #print "dim of leftover IntAlloy: ", np.sum(final_leftover_IntAlloy,axis=1,keepdims=True).shape,"\n"
    #print "IntAlloy utilization ratio (utilized/leftover): ",final_utilized_IntAlloy/final_leftover_IntAlloy,"\n"
    #see percentage of intermediate scrap alloy utlized for alloy production 
    #print ("IntAlloy utilization percent (utilized/(utilized+leftover) by vehicle type): ", final_utilized_ratio,"\n") #(y,c,t)
    #see percentage of alloy demand offset by using secondary material (with virgin bulk and AE)
    #print "Offset by secondary-derived alloys (offset/(offset+still_needed): ",final_virgin_alloy_offset/(final_virgin_alloy_offset+final_virgin_alloy_needed),"\n"
    
    ###emissions
    #print "normalized LC emissions (kg CO2-eq/veh): ",final_norm_LC_emissions,"\n"
    
    ###energy consumption
    #print "normalized LC energy (MJ/veh): ",final_norm_LC_energy,"\n"
    
    """export results"""
    ###export scrap reuse ratios, reshaped to (y*c, t)    
    final_utilized_ratio_reshaped=final_utilized_ratio.reshape((-1,final_utilized_ratio.shape[2])) #(y*c, t), no need to do "order='F' "
    np.savetxt("scrap utilization ratios_CSV.csv", final_utilized_ratio_reshaped, delimiter=",")
    
    ###export energy consumption, reshape to (x*c,t)
    final_norm_LC_energy_reshaped=final_norm_LC_energy.reshape((-1,final_norm_LC_energy.shape[2])) #(x*c,t), if no [min,max] combination is investigated, x (axis=0) should be 1
    np.savetxt("normalized LC energy (MJ per veh)_CSV.csv",final_norm_LC_energy_reshaped, delimiter=",")
    
    
    ###export GHG emissions, reshape to (x*c,t)
    final_norm_LC_emissions_reshaped=final_norm_LC_emissions.reshape((-1,final_norm_LC_emissions.shape[2])) #(x*c,t), if no [min,max] combination is investigated, x (axis=0) should be 1
    np.savetxt("normalized LC emissions (kg CO2-eq per veh)_CSV.csv",final_norm_LC_emissions_reshaped,delimiter=",")
    
       
    return final_utilized_ratio
    

In [ ]:
"""Test the Wrapper module"""
##====dummy data====##
#v_c_tensor_dummy=np.ones((4,1,3))*1000 #4 components, 3 vehicle types [now an intermediate variable converted from Data parsing]
v_s_tensor_dummy=np.ones((7,1,6)) #dummy data, but REAL DIMENSIONS: 7 components, 6 vehicles
#mat_compnt_tensor_dummy=np.ones((5,4,3))*2 #5 alloys, 4 components, 3 vehicles [now an intermediate variable output from Data parsing]
#AE_matrix_dummy=np.arange(360).reshape((60,6)) #60 alloys, 12 AE types [now an intermediate variable output from Data parsing]
shredding_loss_rate_dummy=[0.075,0.075,0.075,0.075,0.075,0.075,0.075] #REAL data,REAL DIMENSIONS: 7 components
#shredding_loss_rate_dummy=[0.0,0.0,0.0,0.0,0.0,0.0,0.0] #TEST no shredding loss
alloy_cutoff_dummy=[(0,3),(4,5),(6,57),(58,59)] #THIS IS REAL DATA FOR ALLOY (BY METAL GROUP) CUTOFF in alloy-related tensors (dim_nc)
                                                #[steel,Fe,Al,Mg] sequence for alloy by metal group
#TD_matrix_dummy=np.linspace(0.1,1.0,48).reshape(4,12) #dummy data, but REAL DIMENSIONS: 4 metal groups, 12 AE types
#TargetAlloy_mass_by_compnt_dummy=np.random.rand(5,6,4)*10000 #5 alloys, 6 AE types, 4 components [now an intermediate variable converted from Data par
compnt_loc_tuple_list=[(2,61), (63,122),(124,183),(185,244),(246,305),(307,366),(368,427)] #THIS IS REAL DATA for vehicle composition
MFA_time_series_file='Vehicle flows_input_for_VehiReLCA_beta1.2.xlsx'
scenario_num=2
metal_process_data_workbook='Data_input_for_VehiReLCA_beta1.2.xlsx'

body_composition_workbook='Composition_input_for_VehiReLCA_beta1.2.xlsx'
time_span_tuple=(2016,2050)
#Depreciated, bulk_metal_row_list=[142,142,140,145] #REAL data for bulk metals made from primary source [steel,Fe,Al,Mg]
                       #CAUTION: 1st and 2nd elements of this list both refer to 'iron primary', as steel group (1st element) also requires iron as bulk metal
"""ALSO NEED TO DEAL WITH two Mg primary sources!!! (need for both bulk and AE)"""
#solution: assign all demand to the row of "Electrolytic" first, then use an exogenous variable to allocate part of the demand to 'Pigdeon'
    
AE_row_list=[140,141,142,143,144,145,147,148,149,150,151,152] #REAL data for AE, make sure the sequence of row# in this list matches the sequence of AE in columns
                                #[Al,Si,Fe,Cu,Mn,Mg,Cr,Ni,Zn,Ti,Mo,C]
Mg_Pigd_share_dict={146:0.8} #dummy data for 'Pigdeon' share, but REAL row#

remelt_scrap_row_list=[153,156,154,155] #REAL data for remelted scrap, make sure the sequence of row# in this list matches the sequence of scrap remelted in PIOT
                                        #[steel,Fe,Al,Mg] sequence for scrap remelted

#input variables for new optimization-based blending func
dummy_alpha=15 #dummy data, seems to work well (at least in testing of the func itself)
dummy_num_iter=100 #dummy data, seems to work well (at least in testing of the func itself)
dummy_epsilon=0.005 #dummy data, seems to work well (at least in testing of the func itself)
dummy_total_utility_score_goal=0.01 #dummy data, seems to work well (at least in testing of the func itself)

#input variable for closed-loop recycling rate factors
#closed_loop_recycl_rate_list=[0.5,1,0.5,1] #dummy data for closed-loop recycling rate factors
#closed_loop_recycl_rate_list=[0,0,0,0] #test zero utilization case
closed_loop_recycl_rate_list=[0.26,1,0.75,0.52] #GREET data for closed-loop recycling rate factors
                                             #0.26 for steel, 1 (assumed) for iron, 0.75 (assume average between wrought and cast Al), 0.52 for Mg

    
#kwargs
assembly_energy_consumption_MJ_vehicle=8340 #value from GREET2 'Vehi_inputs' tab, MJ/vehi
assembly_emissions_kgCO2eq_vehicle=0.127*8340 #assume 100% electricity-powered assemblying process, EF of electricity is the same as what's used for other LCA steps
    
"""test run (cross-mixing)
final_utilized_ratio=Wrapper_MOD(MFA_time_series_file,scenario_num,metal_process_data_workbook,body_composition_workbook,time_span_tuple,compnt_loc_tuple_list,
                v_s_tensor_dummy,shredding_loss_rate_dummy,
            alloy_cutoff_dummy,bulk_metal_row_list,AE_row_list,
            Mg_Pigd_share_dict,remelt_scrap_row_list,dummy_alpha,dummy_num_iter,dummy_epsilon,dummy_total_utility_score_goal,closed_loop_recycl_rate_list,
            max_or_min='min',dilution_explicit=True,rand_sort_error=0.0,cross_mixing_flag=True,assembly_energy_consumption_MJ_vehicle=0.008,
            assembly_emissions_kgCO2eq_vehicle=0.127*0.008)"""


"""test run (no cross-mixing)"""
final_utilized_ratio=Wrapper_MOD(MFA_time_series_file,scenario_num,metal_process_data_workbook,body_composition_workbook,time_span_tuple,compnt_loc_tuple_list,
                v_s_tensor_dummy,shredding_loss_rate_dummy,
            alloy_cutoff_dummy,AE_row_list,
            Mg_Pigd_share_dict,remelt_scrap_row_list,dummy_alpha,dummy_num_iter,dummy_epsilon,dummy_total_utility_score_goal,closed_loop_recycl_rate_list,
            max_or_min='min',dilution_explicit=True,rand_sort_error=0.02,
                cross_mixing_flag=True,assembly_energy_consumption_MJ_vehicle=8340,
            assembly_emissions_kgCO2eq_vehicle=0.127*8340)


